2. Momentum Strategy

3. MACD Crossover Strategy: https://in.tradingview.com/script/pQdJoFHS-Multi-Indicator-Trading-Bot/

5. EMA Pullback Speed Strategy : https://in.tradingview.com/script/cxhQ5d5x-EMA-Pullback-Speed-Strategy/

6. Volatility Based Model: https://in.tradingview.com/script/Lyi4p6th-Volatility-Bias-Model/

7. Price Statistical Strategy: https://in.tradingview.com/script/km8PhlR8-Price-Statistical-Strategy-Z-Score-V-1-01/

9. Random State Machine Strategy: https://in.tradingview.com/script/utM873Jx-Random-State-Machine-Strategy/

10. Reversal Trap Sniper: https://in.tradingview.com/script/4b2zWsyX-Reversal-Trap-Sniper-Verified-Version/


Strats Done:
1. RSI Divergence Strategy: https://in.tradingview.com/script/8GoIeone-RSI-Divergence-Strategy/

4. Long/Short/Exit/Risk management Strategy:  https://in.tradingview.com/script/JGS22oaP-Long-Short-Exit-Risk-management-Strategy/

8. SIGMA Stack Strategy: https://in.tradingview.com/script/2Q5tFJUc-MACD-RSI-EMA-BB-ATR-Day-Trading-Strategy/

In [ ]:
!pip install numpy==1.24.4 --force-reinstall

  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
xarray-einstats 0.9.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.4.0 requi

In [ ]:
!pip install --upgrade pandas-ta
!pip install backtesting

# 1. Original RSI Divergence Strategy

In [ ]:
import pandas as pd
import numpy as np
from backtesting import Strategy
from backtesting.lib import crossover

# It's common to use pandas_ta or talib for indicators in backtesting.py
# If not installed, you might need: pip install pandas_ta TA-Lib
try:
    import pandas_ta as ta
except ImportError:
    print("pandas_ta not found. Please install it with 'pip install pandas_ta'")
    # You might want to exit or handle this more gracefully
    ta = None

class RsiDivergenceStrategy(Strategy):
    # Define parameters based on Pine Script inputs
    rsiLength = 14
    srsiLength = 14
    kLength = 3
    dLength = 3
    emaLength = 200
    lookback = 40
    minSwingDistPercent = 1.5
    minPriceMovePercent = 0.5

    def init(self):
        if ta is None:
            raise ImportError("pandas_ta is required but not installed.")

        # Calculate indicators
        self.rsi = self.I(ta.rsi, self.data.Close, self.rsiLength)
        # In Pine Script, stoch(src, high, low, length)
        # For SRSI, the 'src' is RSI itself.
        # pandas_ta's stoch calculates %K and %D. We need %K for SRSI logic.
        # ta.stoch takes high, low, close. So, we'll use RSI for all three
        # and then calculate SMA for %K.
        stoch_rsi_k, _ = self.I(ta.stoch, self.rsi, self.rsi, self.rsi, self.srsiLength, append=True)
        self.srsi_k = self.I(pd.Series, ta.sma(pd.Series(stoch_rsi_k), self.kLength), name="SRSI_K")
        self.ema200 = self.I(ta.ema, self.data.Close, self.emaLength)

        # Initialize variables for divergence detection
        self.lastLowPrice = -1.0
        self.lastLowRsi = -1.0
        self.lastLowSrsi = -1.0
        self.lastLowIndex = -1

        self.lastHighPrice = -1.0
        self.lastHighRsi = -1.0
        self.lastHighSrsi = -1.0
        self.lastHighIndex = -1

    def next(self):
        current_low = self.data.Low[-1]
        current_high = self.data.High[-1]
        current_rsi = self.rsi[-1]
        current_srsi_k = self.srsi_k[-1]
        current_close = self.data.Close[-1]

        # Find lowest/highest bar in lookback period
        # Using numpy.argmin/argmax on a slice of data
        lookback_lows = self.data.Low[-self.lookback:]
        lookback_highs = self.data.High[-self.lookback:]

        is_lowest_in_lookback = False
        if len(lookback_lows) == self.lookback and lookback_lows[-1] == np.min(lookback_lows):
            is_lowest_in_lookback = True

        is_highest_in_lookback = False
        if len(lookback_highs) == self.lookback and lookback_highs[-1] == np.max(lookback_highs):
            is_highest_in_lookback = True

        bullishDiv = False
        bearishDiv = False
        bullishHiddenDiv = False
        bearishHiddenDiv = False

        # === Bullish Regular Divergence ===
        if is_lowest_in_lookback:
            if self.lastLowPrice != -1.0 and self.lastLowRsi != -1.0 and self.lastLowSrsi != -1.0:
                swingDistLow = abs(current_low - self.lastLowPrice) / self.lastLowPrice * 100
                priceMoveLow = abs(current_low - self.lastLowPrice) / self.lastLowPrice * 100

                if swingDistLow >= self.minSwingDistPercent and priceMoveLow >= self.minPriceMovePercent:
                    if (current_low < self.lastLowPrice and current_rsi > self.lastLowRsi) or \
                       (current_low < self.lastLowPrice and current_srsi_k > self.lastLowSrsi):
                        bullishDiv = True

            # Update last low for next iteration
            self.lastLowPrice = current_low
            self.lastLowRsi = current_rsi
            self.lastLowSrsi = current_srsi_k
            self.lastLowIndex = len(self.data.Close) - 1 # Current index


        # === Bearish Regular Divergence ===
        if is_highest_in_lookback:
            if self.lastHighPrice != -1.0 and self.lastHighRsi != -1.0 and self.lastHighSrsi != -1.0:
                swingDistHigh = abs(current_high - self.lastHighPrice) / self.lastHighPrice * 100
                priceMoveHigh = abs(current_high - self.lastHighPrice) / self.lastHighPrice * 100

                if swingDistHigh >= self.minSwingDistPercent and priceMoveHigh >= self.minPriceMovePercent:
                    if (current_high > self.lastHighPrice and current_rsi < self.lastHighRsi) or \
                       (current_high > self.lastHighPrice and current_srsi_k < self.lastHighSrsi):
                        bearishDiv = True

            # Update last high for next iteration
            self.lastHighPrice = current_high
            self.lastHighRsi = current_rsi
            self.lastHighSrsi = current_srsi_k
            self.lastHighIndex = len(self.data.Close) - 1 # Current index

        # === Bullish Hidden Divergence ===
        if is_lowest_in_lookback: # Current bar is a new low
            # For hidden divergence, compare current higher low with previous lower low
            if self.lastLowPrice != -1.0 and self.lastLowRsi != -1.0 and self.lastLowSrsi != -1.0:
                swingDistLowHidden = abs(current_low - self.lastLowPrice) / self.lastLowPrice * 100
                priceMoveLowHidden = abs(current_low - self.lastLowPrice) / self.lastLowPrice * 100

                if swingDistLowHidden >= self.minSwingDistPercent and priceMoveLowHidden >= self.minPriceMovePercent:
                    if (current_low > self.lastLowPrice and current_rsi < self.lastLowRsi) or \
                       (current_low > self.lastLowPrice and current_srsi_k < self.lastLowSrsi):
                        bullishHiddenDiv = True

        # === Bearish Hidden Divergence ===
        if is_highest_in_lookback: # Current bar is a new high
            # For hidden divergence, compare current lower high with previous higher high
            if self.lastHighPrice != -1.0 and self.lastHighRsi != -1.0 and self.lastHighSrsi != -1.0:
                swingDistHighHidden = abs(current_high - self.lastHighPrice) / self.lastHighPrice * 100
                priceMoveHighHidden = abs(current_high - self.lastHighPrice) / self.lastHighPrice * 100

                if swingDistHighHidden >= self.minSwingDistPercent and priceMoveHighHidden >= self.minPriceMovePercent:
                    if (current_high < self.lastHighPrice and current_rsi > self.lastHighRsi) or \
                       (current_high < self.lastHighPrice and current_srsi_k > self.lastHighSrsi):
                        bearishHiddenDiv = True


        # Strategy entries
        if bullishDiv or bullishHiddenDiv:
            self.buy() # Equivalent to strategy.entry("Long", strategy.long)

        if bearishDiv or bearishHiddenDiv:
            self.sell() # Equivalent to strategy.entry("Short", strategy.short)

        # Plotting EMA (optional, for visualization in backtesting)
        # self.plot(self.ema200, color='purple', overlay=True, name="EMA 200")

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


# Improved RSI Divergence Strategy

In [ ]:
import pandas as pd
import numpy as np
from backtesting import Strategy
from backtesting.lib import crossover

# Ensure pandas_ta is installed for indicator calculations
try:
    import pandas_ta as ta
except ImportError:
    print("pandas_ta not found. Please install it with 'pip install pandas_ta'")
    # You might want to exit or handle this more gracefully, or provide a mock ta for testing
    ta = None

class ImprovedRsiDivergenceStrategy(Strategy):
    """
    An improved RSI and Stochastic RSI divergence strategy with EMA trend
    filter, overbought/oversold zone confirmation, stricter divergence
    conditions, and integrated Stop Loss/Take Profit.
    """

    # --- Strategy Parameters (Optimizable) ---
    rsiLength = 14              # Length for RSI calculation
    srsiLength = 14             # Length for Stochastic RSI calculation
    kLength = 3                 # %K length for Stochastic RSI
    dLength = 3                 # %D length for Stochastic RSI (not directly used for divergence here, but for completeness)
    emaLength = 200             # Length for Exponential Moving Average (trend filter)
    lookback = 40               # Lookback period for identifying swing highs/lows for divergence
    minSwingDistPercent = 1.5   # Minimum percentage distance between swing points for a valid divergence
    minPriceMovePercent = 0.5   # Minimum percentage price move from last swing point
    stopLossPercent = 2.0       # Percentage for Stop Loss (e.g., 2.0 means 2% below/above entry)
    takeProfitPercent = 4.0     # Percentage for Take Profit (e.g., 4.0 means 4% above/below entry)

    # Overbought/Oversold thresholds for RSI and Stochastic RSI
    rsiOversoldThreshold = 30
    rsiOverboughtThreshold = 70
    srsiOversoldThreshold = 20  # Typical for Stochastics (0-100 scale)
    srsiOverboughtThreshold = 80 # Typical for Stochastics (0-100 scale)

    def init(self):
        """
        Initializes the strategy, calculates indicators, and sets up
        variables for tracking divergence.
        """
        if ta is None:
            raise ImportError("pandas_ta is required but not installed. Please install it with 'pip install pandas_ta'")

        # Calculate indicators using self.I for backtesting.py integration
        self.rsi = self.I(ta.rsi, self.data.Close, self.rsiLength, name="RSI")
        # Stochastic RSI uses RSI as its source (src)
        # ta.stoch returns %K and %D. We need %K for divergence.
        # It expects high, low, close. We feed it the RSI values for all three.
        stoch_rsi_k, _ = self.I(ta.stoch, self.rsi, self.rsi, self.rsi, self.srsiLength, append=True, name="SRSI_K_RAW")
        # Then calculate SMA for the %K of SRSI
        self.srsi_k = self.I(pd.Series, ta.sma(pd.Series(stoch_rsi_k), self.kLength), name="SRSI_K")
        self.ema200 = self.I(ta.ema, self.data.Close, self.emaLength, name="EMA200")

        # Variables to store the last significant swing points and their indicator values
        # Initialized to -1.0 to indicate 'not set' or 'invalid'
        self.lastLowPrice = -1.0
        self.lastLowRsi = -1.0
        self.lastLowSrsi = -1.0
        self.lastLowIndex = -1 # Not strictly used for logic, but helpful for debugging/plotting concepts

        self.lastHighPrice = -1.0
        self.lastHighRsi = -1.0
        self.lastHighSrsi = -1.0
        self.lastHighIndex = -1 # Not strictly used for logic, but helpful for debugging/plotting concepts

    def next(self):
        """
        Called on each new bar (candlestick) to evaluate trading conditions.
        """
        # Ensure enough data points are available for calculations
        if len(self.data.Close) < max(self.emaLength, self.rsiLength, self.srsiLength, self.lookback):
            return

        # Get current bar's values
        current_low = self.data.Low[-1]
        current_high = self.data.High[-1]
        current_close = self.data.Close[-1]
        current_rsi = self.rsi[-1]
        current_srsi_k = self.srsi_k[-1]

        # Check for sufficient lookback data for min/max calculations
        if len(self.data.Low) < self.lookback or len(self.data.High) < self.lookback:
            return # Not enough data for lookback period, skip current bar

        # Identify if current bar is the lowest/highest in the lookback period
        is_lowest_in_lookback = False
        if current_low == np.min(self.data.Low[-self.lookback:]):
            is_lowest_in_lookback = True

        is_highest_in_lookback = False
        if current_high == np.max(self.data.High[-self.lookback:]):
            is_highest_in_lookback = True

        # Initialize divergence flags for the current bar
        bullishDiv = False
        bearishDiv = False
        bullishHiddenDiv = False
        bearishHiddenDiv = False

        # --- Bullish Regular Divergence (Price lower, Indicator higher) ---
        # Current bar is a new low within the lookback period
        if is_lowest_in_lookback:
            # Ensure we have a previous valid low to compare with
            if self.lastLowPrice != -1.0 and self.lastLowRsi != -1.0 and self.lastLowSrsi != -1.0:
                # Check swing distance and price move filters
                swingDistLow = abs(current_low - self.lastLowPrice) / self.lastLowPrice * 100
                priceMoveLow = abs(current_low - self.lastLowPrice) / self.lastLowPrice * 100

                if swingDistLow >= self.minSwingDistPercent and priceMoveLow >= self.minPriceMovePercent:
                    # Stricter condition: BOTH RSI and SRSI must show divergence
                    if (current_low < self.lastLowPrice and current_rsi > self.lastLowRsi) and \
                       (current_low < self.lastLowPrice and current_srsi_k > self.lastLowSrsi):
                        bullishDiv = True

            # Always update the last low point if current bar is a new low
            # This ensures we always track the most recent 'lowest-in-lookback' point
            self.lastLowPrice = current_low
            self.lastLowRsi = current_rsi
            self.lastLowSrsi = current_srsi_k
            self.lastLowIndex = len(self.data.Close) - 1 # Store current index

        # --- Bearish Regular Divergence (Price higher, Indicator lower) ---
        # Current bar is a new high within the lookback period
        if is_highest_in_lookback:
            # Ensure we have a previous valid high to compare with
            if self.lastHighPrice != -1.0 and self.lastHighRsi != -1.0 and self.lastHighSrsi != -1.0:
                # Check swing distance and price move filters
                swingDistHigh = abs(current_high - self.lastHighPrice) / self.lastHighPrice * 100
                priceMoveHigh = abs(current_high - self.lastHighPrice) / self.lastHighPrice * 100

                if swingDistHigh >= self.minSwingDistPercent and priceMoveHigh >= self.minPriceMovePercent:
                    # Stricter condition: BOTH RSI and SRSI must show divergence
                    if (current_high > self.lastHighPrice and current_rsi < self.lastHighRsi) and \
                       (current_high > self.lastHighPrice and current_srsi_k < self.lastHighSrsi):
                        bearishDiv = True

            # Always update the last high point if current bar is a new high
            self.lastHighPrice = current_high
            self.lastHighRsi = current_rsi
            self.lastHighSrsi = current_srsi_k
            self.lastHighIndex = len(self.data.Close) - 1 # Store current index

        # --- Bullish Hidden Divergence (Price higher low, Indicator lower low) ---
        # Current bar is a new low within the lookback period, indicating a potential swing point
        if is_lowest_in_lookback:
            if self.lastLowPrice != -1.0 and self.lastLowRsi != -1.0 and self.lastLowSrsi != -1.0:
                swingDistLowHidden = abs(current_low - self.lastLowPrice) / self.lastLowPrice * 100
                priceMoveLowHidden = abs(current_low - self.lastLowPrice) / self.lastLowPrice * 100

                if swingDistLowHidden >= self.minSwingDistPercent and priceMoveLowHidden >= self.minPriceMovePercent:
                    # Stricter condition: BOTH RSI and SRSI must show divergence
                    if (current_low > self.lastLowPrice and current_rsi < self.lastLowRsi) and \
                       (current_low > self.lastLowPrice and current_srsi_k < self.lastLowSrsi):
                        bullishHiddenDiv = True

        # --- Bearish Hidden Divergence (Price lower high, Indicator higher high) ---
        # Current bar is a new high within the lookback period, indicating a potential swing point
        if is_highest_in_lookback:
            if self.lastHighPrice != -1.0 and self.lastHighRsi != -1.0 and self.lastHighSrsi != -1.0:
                swingDistHighHidden = abs(current_high - self.lastHighPrice) / self.lastHighPrice * 100
                priceMoveHighHidden = abs(current_high - self.lastHighPrice) / self.lastHighPrice * 100

                if swingDistHighHidden >= self.minSwingDistPercent and priceMoveHighHidden >= self.minPriceMovePercent:
                    # Stricter condition: BOTH RSI and SRSI must show divergence
                    if (current_high < self.lastHighPrice and current_rsi > self.lastHighRsi) and \
                       (current_high < self.lastHighPrice and current_srsi_k > self.lastHighSrsi):
                        bearishHiddenDiv = True

        # --- Strategy Entry Logic with Filters ---

        # 1. Bullish Entry Conditions
        # Check for a buy signal ONLY if no position is currently open
        if not self.position:
            # Trend filter: Price above EMA
            is_uptrend = current_close > self.ema200[-1]
            # Overbought/Oversold filter: Indicators in oversold territory
            rsi_oversold = current_rsi < self.rsiOversoldThreshold
            srsi_k_oversold = current_srsi_k < self.srsiOversoldThreshold

            if (bullishDiv or bullishHiddenDiv) and is_uptrend and rsi_oversold and srsi_k_oversold:
                self.buy()
                # Set Stop Loss and Take Profit levels
                self.stop_loss = current_close * (1 - self.stopLossPercent / 100)
                self.take_profit = current_close * (1 + self.takeProfitPercent / 100)

        # 2. Bearish Entry Conditions
        # Check for a sell signal ONLY if no position is currently open
        if not self.position:
            # Trend filter: Price below EMA
            is_downtrend = current_close < self.ema200[-1]
            # Overbought/Oversold filter: Indicators in overbought territory
            rsi_overbought = current_rsi > self.rsiOverboughtThreshold
            srsi_k_overbought = current_srsi_k > self.srsiOverboughtThreshold

            if (bearishDiv or bearishHiddenDiv) and is_downtrend and rsi_overbought and srsi_k_overbought:
                self.sell()
                # Set Stop Loss and Take Profit levels
                self.stop_loss = current_close * (1 + self.stopLossPercent / 100)
                self.take_profit = current_close * (1 - self.takeProfitPercent / 100)

# 2. Momentum Strategy

# Improved Momentum Strategy

# 3. MACD Strategy

In [ ]:
# user_strategy.py

from backtesting import Strategy
from backtesting.lib import crossover
import pandas_ta as ta

class MultiIndicatorStrategy(Strategy):
    """
    A multi-indicator trading strategy translated from a Pine Script original.

    This strategy combines signals from four different technical indicators:
    1. Simple Moving Average (SMA) Crossover
    2. Relative Strength Index (RSI)
    3. Moving Average Convergence Divergence (MACD)
    4. Bollinger Bands (BB)

    A trade is entered when a minimum number of these indicators give a
    simultaneous bullish or bearish signal. The position size is dynamically
    calculated based on a user-defined risk percentage of the total equity.
    An optional stop-loss can be used for risk management.
    """

    # ===== INPUT PARAMETERS =====
    # These can be optimized by passing them as arguments to bt.run()

    # Risk Management
    risk_per_trade_pct = 2.0      # Risk Per Trade (%)
    max_position_size_pct = 10.0  # Max Position Size (%)
    use_stop_loss = True
    stop_loss_pct = 2.0           # Stop Loss (%)

    # Technical Indicator Parameters
    sma_short_period = 20
    sma_long_period = 50
    rsi_period = 14
    rsi_oversold = 30
    rsi_overbought = 70
    macd_fast = 12
    macd_slow = 26
    macd_signal = 9
    bb_length = 20
    bb_mult = 2.0

    # Signal Threshold
    min_signals = 2

    def init(self):
        """
        Initializes the strategy by calculating all the necessary indicators.
        The self.I() function is used to wrap indicator calculations, which
        backtesting.py then manages for caching and plotting.
        """
        # Use pandas_ta to calculate indicators
        self.df = self.data.to_df() # Convert backtesting.py data to pandas DataFrame

        # SMA
        self.df.ta.sma(length=self.sma_short_period, append=True)
        self.sma_short_val = self.df[f'SMA_{self.sma_short_period}']

        self.df.ta.sma(length=self.sma_long_period, append=True)
        self.sma_long_val = self.df[f'SMA_{self.sma_long_period}']

        # RSI
        self.df.ta.rsi(length=self.rsi_period, append=True)
        self.rsi_val = self.df[f'RSI_{self.rsi_period}']

        # MACD
        self.df.ta.macd(fast=self.macd_fast, slow=self.macd_slow, signal=self.macd_signal, append=True)
        self.macd_line = self.df[f'MACD_{self.macd_fast}_{self.macd_slow}_{self.macd_signal}']
        self.signal_line = self.df[f'MACDs_{self.macd_fast}_{self.macd_slow}_{self.macd_signal}']

        # Bollinger Bands
        self.df.ta.bbands(length=self.bb_length, std=self.bb_mult, append=True)
        self.bb_lower = self.df[f'BBL_{self.bb_length}_{self.bb_mult}']
        self.bb_upper = self.df[f'BBU_{self.bb_length}_{self.bb_mult}']


    def next(self):
        """
        The core logic of the strategy, executed for each candlestick.
        """
        # Get the most recent price
        price = self.data.Close[-1]

        # ===== SIGNAL GENERATION =====
        # Moving Average Crossover Signals
        ma_cross_up = crossover(self.sma_short_val, self.sma_long_val)
        ma_cross_down = crossover(self.sma_long_val, self.sma_short_val)

        # RSI Signals
        rsi_oversold_signal = self.rsi_val[-1] < self.rsi_oversold
        rsi_overbought_signal = self.rsi_val[-1] > self.rsi_overbought

        # MACD Signals
        macd_bull_cross = crossover(self.macd_line, self.signal_line)
        macd_bear_cross = crossover(self.signal_line, self.macd_line)

        # Bollinger Bands Signals
        bb_lower_touch = price < self.bb_lower[-1]
        bb_upper_touch = price > self.bb_upper[-1]

        # ===== SIGNAL COUNTING =====
        # Count bullish signals
        bullish_signals = (
            int(ma_cross_up) +
            int(rsi_oversold_signal) +
            int(macd_bull_cross) +
            int(bb_lower_touch)
        )

        # Count bearish signals
        bearish_signals = (
            int(ma_cross_down) +
            int(rsi_overbought_signal) +
            int(macd_bear_cross) +
            int(bb_upper_touch)
        )

        # ===== TRADING LOGIC =====
        # Entry conditions
        long_condition = bullish_signals >= self.min_signals and bullish_signals > bearish_signals
        short_condition = bearish_signals >= self.min_signals and bearish_signals > bullish_signals

        # Exit conditions (close open position on opposing signal)
        if self.position.is_long and short_condition:
            self.position.close()

        if self.position.is_short and long_condition:
            self.position.close()

        # Entry orders (only if no position is currently open)
        if not self.position:
            # Position size calculation
            size = 0
            if self.use_stop_loss and self.stop_loss_pct > 0:
                # Risk-based sizing: Position Value = (Total Equity * % Risk) / (% Stop Loss)
                position_value = self.equity * (self.risk_per_trade_pct / 100) / (self.stop_loss_pct / 100)
            else:
                # Fallback to max position size if SL is not used
                position_value = self.equity * (self.max_position_size_pct / 100)

            # Ensure position size does not exceed the maximum allowed
            max_value_allowed = self.equity * (self.max_position_size_pct / 100)
            final_position_value = min(position_value, max_value_allowed)

            # Convert dollar value to a size fraction for backtesting.py's buy/sell 'size' parameter
            size = final_position_value / self.equity

            # Ensure size is between 0 and 1
            size = max(0, min(1, size))

            # Stop loss price calculation
            long_stop_loss = price * (1 - self.stop_loss_pct / 100) if self.use_stop_loss else None
            short_stop_loss = price * (1 + self.stop_loss_pct / 100) if self.use_stop_loss else None

            # Place orders
            if long_condition:
                self.buy(size=size, sl=long_stop_loss)
            elif short_condition:
                self.sell(size=size, sl=short_stop_loss)

# Improved MACD Strategy

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover
import pandas_ta as ta

class ImprovedMultiIndicatorStrategy(Strategy):
    """
    An improved multi-indicator trading strategy with advanced filters and dynamic parameters.

    This version enhances the original strategy by incorporating:
    1. Trend Filtering: Uses a long-period SMA to ensure trades are only taken
       in the direction of the primary trend.
    2. Market Regime Filter (ADX): Avoids trades during non-trending, choppy
       markets to reduce whipsaws.
    3. Dynamic Volatility-Based Stops (ATR): Sets stop-loss levels based on
       the Average True Range (ATR), adapting to market volatility.
    4. Proactive Profit-Taking: Implements a fixed risk-to-reward ratio to
       systematically lock in profits.
    """

    # ===== INPUT PARAMETERS (can be optimized in bt.run()) =====

    # -- Risk & Position Sizing --
    risk_per_trade_pct = 2.0      # The percentage of equity to risk on a single trade.
    max_position_size_pct = 25.0  # The maximum allowable position size as a percentage of equity.

    # -- Stop-Loss Configuration --
    use_atr_stop = True           # Set to True to use ATR for stops, False for percentage-based.
    atr_period = 14               # ATR calculation period.
    atr_multiplier = 2.0          # Multiplier for the ATR value to set the stop distance.
    stop_loss_pct = 3.0           # Fallback stop-loss percentage if use_atr_stop is False.

    # -- Take-Profit Configuration --
    risk_reward_ratio = 1.5       # The ratio of reward to risk (e.g., 1.5 means TP is 1.5x the SL distance).

    # -- Trend & Regime Filters --
    sma_long_period = 200         # Period for the long-term trend-filtering SMA.
    adx_period = 14               # Period for the ADX calculation.
    adx_threshold = 25            # ADX value above which the market is considered trending.

    # -- Core Indicator Parameters --
    sma_short_period = 20
    rsi_period = 14
    rsi_oversold = 35
    rsi_overbought = 65
    macd_fast = 12
    macd_slow = 26
    macd_signal = 9
    bb_length = 20
    bb_mult = 2.0

    # -- Signal Threshold --
    min_signals = 2               # Minimum number of confirming signals required for entry.


    def init(self):
        """
        Initializes the strategy by calculating all necessary indicators.
        """
        # For ease of use, convert backtesting.py data to a pandas DataFrame
        self.df = self.data.to_df()

        # --- Calculate all indicators using pandas_ta ---

        # Core entry indicators
        self.df.ta.sma(length=self.sma_short_period, append=True)
        self.sma_short_val = self.df[f'SMA_{self.sma_short_period}']

        self.df.ta.rsi(length=self.rsi_period, append=True)
        self.rsi_val = self.df[f'RSI_{self.rsi_period}']

        self.df.ta.macd(fast=self.macd_fast, slow=self.macd_slow, signal=self.macd_signal, append=True)
        self.macd_line = self.df[f'MACD_{self.macd_fast}_{self.slow}_{self.signal}']
        self.signal_line = self.df[f'MACDs_{self.macd_fast}_{self.slow}_{self.signal}']

        self.df.ta.bbands(length=self.bb_length, std=self.bb_mult, append=True)
        self.bb_lower = self.df[f'BBL_{self.bb_length}_{self.bb_mult}']
        self.bb_upper = self.df[f'BBU_{self.bb_length}_{self.bb_mult}']

        # Filter indicators
        self.df.ta.sma(length=self.sma_long_period, append=True)
        self.sma_long_val = self.df[f'SMA_{self.sma_long_period}']

        self.df.ta.adx(length=self.adx_period, append=True)
        self.adx = self.df[f'ADX_{self.adx_period}']

        # Stop-loss indicator
        self.df.ta.atr(length=self.atr_period, append=True)
        self.atr = self.df[f'ATRr_{self.atr_period}']


    def next(self):
        """
        The core logic of the strategy, executed for each candlestick.
        """
        price = self.data.Close[-1]

        # ===== 1. DEFINE MARKET REGIME & TREND =====
        is_trending = self.adx[-1] > self.adx_threshold
        is_uptrend = price > self.sma_long_val[-1]
        is_downtrend = price < self.sma_long_val[-1]

        # ===== 2. GENERATE CORE ENTRY SIGNALS =====
        # Note: MA Crossover is no longer used as a direct signal.

        # RSI Signals
        rsi_oversold_signal = self.rsi_val[-1] < self.rsi_oversold
        rsi_overbought_signal = self.rsi_val[-1] > self.rsi_overbought

        # MACD Signals
        macd_bull_cross = crossover(self.macd_line, self.signal_line)
        macd_bear_cross = crossover(self.signal_line, self.macd_line)

        # Bollinger Bands Signals
        bb_lower_touch = price < self.bb_lower[-1]
        bb_upper_touch = price > self.bb_upper[-1]

        # Price crossing short SMA
        price_cross_sma_short_up = crossover(self.data.Close, self.sma_short_val)

        # ===== 3. COUNT SIGNALS =====
        bullish_signals = (
            int(rsi_oversold_signal) +
            int(macd_bull_cross) +
            int(bb_lower_touch) +
            int(price_cross_sma_short_up)
        )

        bearish_signals = (
            int(rsi_overbought_signal) +
            int(macd_bear_cross) +
            int(bb_upper_touch)
        )

        # ===== 4. APPLY FILTERS TO GET FINAL TRADE CONDITION =====
        # A long requires a trending market, an overall uptrend, and enough bullish signals.
        long_condition = (
            is_trending and
            is_uptrend and
            bullish_signals >= self.min_signals
        )
        # A short requires a trending market, an overall downtrend, and enough bearish signals.
        short_condition = (
            is_trending and
            is_downtrend and
            bearish_signals >= self.min_signals
        )

        # ===== 5. MANAGE EXITS =====
        # Exit long if a valid short condition appears
        if self.position.is_long and short_condition:
            self.position.close(comment="Exit Long on opposing signal")

        # Exit short if a valid long condition appears
        if self.position.is_short and long_condition:
            self.position.close(comment="Exit Short on opposing signal")

        # ===== 6. MANAGE ENTRIES =====
        # Only enter if there is no current position
        if not self.position:
            stop_loss_dist = 0

            # --- Calculate Stop Loss ---
            if self.use_atr_stop:
                stop_loss_dist = self.atr[-1] * self.atr_multiplier
            else:
                stop_loss_dist = price * (self.stop_loss_pct / 100)

            if stop_loss_dist == 0: # Avoid division by zero
                return

            # --- Calculate Position Size ---
            risk_amount = self.equity * (self.risk_per_trade_pct / 100)

            # Position size in units/shares
            position_qty = risk_amount / stop_loss_dist

            # Position size as a fraction of equity for backtesting.py
            size = (position_qty * price) / self.equity

            # Ensure size respects the maximum allowed position
            max_size_allowed = self.max_position_size_pct / 100
            size = min(size, max_size_allowed)

            # Ensure size is a valid fraction (0 to 1)
            size = max(0, min(1, size))

            if size == 0:
                return

            # --- Place Orders ---
            if long_condition:
                sl_price = price - stop_loss_dist
                tp_price = price + stop_loss_dist * self.risk_reward_ratio
                self.buy(size=size, sl=sl_price, tp=tp_price)

            elif short_condition:
                sl_price = price + stop_loss_dist
                tp_price = price - stop_loss_dist * self.risk_reward_ratio
                self.sell(size=size, sl=sl_price, tp=tp_price)

# 4. EMA Pullback Speed Strategy

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover
import numpy as np
import pandas as pd


class EMAPullbackSpeedStrategy(Strategy):
    # === Strategy Parameters ===
    max_length = 50
    accel_multiplier = 3.0
    return_threshold = 5.0  # in %
    atr_length = 14
    atr_mult = 4.0
    fixed_tp_pct = 1.5  # in %
    short_ema_len = 21
    long_ema_len = 50
    long_speed_min = 1000.0
    short_speed_max = -1000.0

    def init(self):
        # EMAs
        self.ema_short = self.I(lambda x: x.ewm(span=self.short_ema_len).mean(), self.data.Close)
        self.ema_long = self.I(lambda x: x.ewm(span=self.long_ema_len).mean(), self.data.Close)

        # ATR
        self.atr = self.I(self.compute_atr, self.data.High, self.data.Low, self.data.Close)

        # Dynamic EMA
        self.dyn_ema = self.I(self.compute_dynamic_ema, self.data.Close)

    def compute_atr(self, high, low, close):
        tr1 = high - low
        tr2 = np.abs(high - close.shift())
        tr3 = np.abs(low - close.shift())
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        return tr.rolling(self.atr_length).mean()

    def compute_dynamic_ema(self, close):
        max_abs_diff = close.rolling(200).apply(lambda x: np.max(np.abs(x - x[-1])), raw=True)
        norm = (close + max_abs_diff) / (2 * max_abs_diff)
        dyn_lengths = 5 + norm * (self.max_length - 5)
        dyn_ema = pd.Series(index=close.index, dtype='float64')
        dyn_ema.iloc[0] = close.iloc[0]

        for i in range(1, len(close)):
            prev_diff = abs(close.iloc[i - 1] - close.iloc[i - 2]) if i >= 2 else 0
            curr_diff = abs(close.iloc[i] - close.iloc[i - 1])
            max_delta = close.iloc[max(0, i - 200):i].diff().abs().max()
            max_delta = max_delta if max_delta != 0 else 1
            accel_factor = curr_diff / max_delta
            alpha_base = 2 / (dyn_lengths.iloc[i] + 1)
            alpha = min(1, alpha_base * (1 + accel_factor * self.accel_multiplier))
            dyn_ema.iloc[i] = alpha * close.iloc[i] + (1 - alpha) * dyn_ema.iloc[i - 1]

        return dyn_ema

    def next(self):
        i = self.data.index[-1]

        close = self.data.Close[i]
        open_ = self.data.Open[i]
        high = self.data.High[i]
        low = self.data.Low[i]

        ema_s = self.ema_short[i]
        ema_l = self.ema_long[i]
        dyn = self.dyn_ema[i]
        atr = self.atr[i]

        # Trend
        is_uptrend = close > dyn
        is_downtrend = close < dyn

        # Return to trend
        distance = abs(close - dyn) / dyn * 100
        returned_to_trend = distance < self.return_threshold

        # EMA Filter
        ema_uptrend = ema_s > ema_l
        ema_downtrend = ema_s < ema_l

        # Candle Reversal Pattern
        bullish_reversal = self.data.Close[i - 2] > self.data.Open[i - 2] and \
                           self.data.Close[i - 1] > self.data.Open[i - 1] and \
                           close > self.data.High[i - 1]

        bearish_reversal = self.data.Close[i - 2] < self.data.Open[i - 2] and \
                            self.data.Close[i - 1] < self.data.Open[i - 1] and \
                            close < self.data.Low[i - 1]

        # Speed
        speed = close - open_
        trend_speed_up = speed > 0
        trend_speed_dn = speed < 0
        trend_speed_filter_long = speed >= self.long_speed_min
        trend_speed_filter_short = speed <= self.short_speed_max

        # Stop Loss / Take Profit
        long_sl = close - atr * self.atr_mult
        short_sl = close + atr * self.atr_mult
        long_tp = close + close * self.fixed_tp_pct / 100
        short_tp = close - close * self.fixed_tp_pct / 100

        # === Entry Conditions ===
        long_condition = all([
            is_uptrend, bullish_reversal, returned_to_trend,
            trend_speed_up, ema_uptrend, trend_speed_filter_long
        ])
        short_condition = all([
            is_downtrend, bearish_reversal, returned_to_trend,
            trend_speed_dn, ema_downtrend, trend_speed_filter_short
        ])

        if long_condition and not self.position:
            self.buy(sl=long_sl, tp=long_tp)

        if short_condition and not self.position:
            self.sell(sl=short_sl, tp=short_tp)

# Improved EMA Pullback Speed Strategy

In [ ]:
from backtesting import Strategy
import numpy as np
import pandas as pd


class ImprovedEMAPullbackSpeedStrategy(Strategy):
    # === Strategy Parameters ===
    max_length = 50
    accel_multiplier = 3.0
    return_threshold = 5.0  # in %
    atr_length = 14
    atr_mult = 2.0
    tp_mult = 2.5
    short_ema_len = 21
    long_ema_len = 50
    normalized_speed_long = 1.5  # In ATR units
    normalized_speed_short = -1.5
    adx_period = 14
    adx_threshold = 20
    volume_filter = True

    def init(self):
        # EMA filters
        self.ema_short = self.I(lambda x: x.ewm(span=self.short_ema_len).mean(), self.data.Close)
        self.ema_long = self.I(lambda x: x.ewm(span=self.long_ema_len).mean(), self.data.Close)

        # ATR
        self.atr = self.I(self.compute_atr, self.data.High, self.data.Low, self.data.Close)

        # Dynamic EMA
        self.dyn_ema = self.I(self.compute_dynamic_ema, self.data.Close)

        # ADX for trend strength
        self.adx = self.I(self.compute_adx, self.data.High, self.data.Low, self.data.Close, self.adx_period)

        # Volume filter
        if self.volume_filter and hasattr(self.data, 'Volume'):
            self.volume_avg = self.I(lambda v: pd.Series(v).rolling(20).mean(), self.data.Volume)

    def compute_atr(self, high, low, close):
        tr1 = high - low
        tr2 = np.abs(high - close.shift())
        tr3 = np.abs(low - close.shift())
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        return tr.rolling(self.atr_length).mean()

    def compute_adx(self, high, low, close, period):
        plus_dm = high.diff()
        minus_dm = low.diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm > 0] = 0
        tr = pd.concat([
            high - low,
            abs(high - close.shift()),
            abs(low - close.shift())
        ], axis=1).max(axis=1)
        tr_smooth = tr.rolling(period).mean()
        plus_di = 100 * (plus_dm.rolling(period).mean() / tr_smooth)
        minus_di = abs(100 * (minus_dm.rolling(period).mean() / tr_smooth))
        dx = 100 * abs(plus_di - minus_di) / (plus_di + minus_di)
        adx = dx.rolling(period).mean()
        return adx

    def compute_dynamic_ema(self, close):
        max_abs_diff = close.rolling(200).apply(lambda x: np.max(np.abs(x - x[-1])), raw=True)
        norm = (close + max_abs_diff) / (2 * max_abs_diff)
        dyn_lengths = 5 + norm * (self.max_length - 5)
        dyn_ema = pd.Series(index=close.index, dtype='float64')
        dyn_ema.iloc[0] = close.iloc[0]

        for i in range(1, len(close)):
            prev_diff = abs(close.iloc[i - 1] - close.iloc[i - 2]) if i >= 2 else 0
            curr_diff = abs(close.iloc[i] - close.iloc[i - 1])
            max_delta = close.iloc[max(0, i - 200):i].diff().abs().max()
            max_delta = max_delta if max_delta != 0 else 1
            accel_factor = curr_diff / max_delta
            alpha_base = 2 / (dyn_lengths.iloc[i] + 1)
            alpha = min(1, alpha_base * (1 + accel_factor * self.accel_multiplier))
            dyn_ema.iloc[i] = alpha * close.iloc[i] + (1 - alpha) * dyn_ema.iloc[i - 1]

        return dyn_ema

    def next(self):
        i = self.data.index[-1]
        if i < 3:  # Not enough bars
            return

        close = self.data.Close[i]
        open_ = self.data.Open[i]
        high = self.data.High[i]
        low = self.data.Low[i]
        atr = self.atr[i]
        dyn = self.dyn_ema[i]
        ema_s = self.ema_short[i]
        ema_l = self.ema_long[i]
        adx = self.adx[i]

        if atr is None or dyn is None or ema_s is None or ema_l is None or adx is None:
            return

        # === Core Conditions ===
        distance = abs(close - dyn) / dyn * 100
        returned_to_trend = distance < self.return_threshold
        ema_uptrend = ema_s > ema_l
        ema_downtrend = ema_s < ema_l
        is_uptrend = close > dyn
        is_downtrend = close < dyn

        # Candle Reversal Patterns (Improved - engulfing)
        bullish_engulfing = self.data.Close[i - 1] < self.data.Open[i - 1] and close > open_ and close > self.data.Open[i - 1] and open_ < self.data.Close[i - 1]
        bearish_engulfing = self.data.Close[i - 1] > self.data.Open[i - 1] and close < open_ and close < self.data.Open[i - 1] and open_ > self.data.Close[i - 1]

        # Normalized Speed
        speed = (close - open_) / atr if atr != 0 else 0
        speed_filter_long = speed >= self.normalized_speed_long
        speed_filter_short = speed <= self.normalized_speed_short

        # Volume Filter
        vol_ok = True
        if self.volume_filter and hasattr(self.data, 'Volume'):
            vol_ok = self.data.Volume[i] > self.volume_avg[i]

        # ADX Filter
        trend_strength = adx >= self.adx_threshold

        # Entry Conditions
        long_condition = all([
            returned_to_trend, is_uptrend, ema_uptrend,
            bullish_engulfing, speed_filter_long,
            trend_strength, vol_ok
        ])

        short_condition = all([
            returned_to_trend, is_downtrend, ema_downtrend,
            bearish_engulfing, speed_filter_short,
            trend_strength, vol_ok
        ])

        # Stop loss and take profit
        long_sl = close - atr * self.atr_mult
        short_sl = close + atr * self.atr_mult
        long_tp = close + atr * self.tp_mult
        short_tp = close - atr * self.tp_mult

        # Execute Orders
        if long_condition and not self.position:
            self.buy(sl=long_sl, tp=long_tp)
        elif short_condition and not self.position:
            self.sell(sl=short_sl, tp=short_tp)

# 5. Volatility Based Model

In [ ]:
import pandas as pd
from backtesting import Strategy
from backtesting.lib import crossover
import pandas_ta as ta

class VolatilityBiasStrategy(Strategy):
    # Strategy Parameters
    bias_window = 10
    bias_threshold = 0.6
    range_min = 0.05
    risk_reward = 2.0
    max_bars = 20
    atr_length = 14

    def init(self):
        # Calculate ATR and store Close prices
        self.atr = self.I(ta.atr, self.data.High, self.data.Low, self.data.Close, length=self.atr_length)

    def next(self):
        if len(self.data.Close) < self.bias_window:
            return

        # === DIRECTIONAL BIAS ===
        up_closes = sum(self.data.Close[-i] > self.data.Open[-i] for i in range(1, self.bias_window + 1))
        bias_ratio = up_closes / self.bias_window

        # === RANGE CHECK ===
        high_range = max(self.data.High[-self.bias_window:])
        low_range = min(self.data.Low[-self.bias_window:])
        range_perc = (high_range - low_range) / low_range

        has_bias_long = bias_ratio >= self.bias_threshold and range_perc > self.range_min
        has_bias_short = bias_ratio <= (1 - self.bias_threshold) and range_perc > self.range_min

        current_close = self.data.Close[-1]
        current_atr = self.atr[-1]

        # === ENTRY CONDITIONS ===
        if has_bias_long and not self.position:
            self.buy()
        elif has_bias_short and not self.position:
            self.sell()

        # === EXIT CONDITIONS ===
        if self.position:
            entry_price = self.position.entry_price
            entry_bar = self.position.entry_bar
            holding_period = len(self.data.Close) - entry_bar

            if self.position.is_long:
                sl = entry_price - current_atr
                tp = entry_price + current_atr * self.risk_reward
                if current_close <= sl or current_close >= tp or holding_period >= self.max_bars:
                    self.position.close()

            elif self.position.is_short:
                sl = entry_price + current_atr
                tp = entry_price - current_atr * self.risk_reward
                if current_close >= sl or current_close <= tp or holding_period >= self.max_bars:
                    self.position.close()

# Improved Volatility Based Model

In [ ]:
import pandas as pd
from backtesting import Strategy
from backtesting.lib import crossover
import pandas_ta as ta

class Improved_Volatility_Strategy(Strategy):
    # Strategy Parameters
    bias_window = 10
    bias_threshold = 0.6
    range_min = 0.05
    risk_reward = 2.0
    max_bars = 20
    atr_length = 14
    volume_ma_len = 20
    adx_threshold = 20
    ema_fast = 20
    ema_slow = 50

    def init(self):
        # Indicators
        self.atr = self.I(ta.atr, self.data.High, self.data.Low, self.data.Close, length=self.atr_length)
        self.rsi = self.I(ta.rsi, self.data.Close, length=14)
        self.adx = self.I(ta.adx, self.data.High, self.data.Low, self.data.Close, length=14)
        self.ema_fast_series = self.I(ta.ema, self.data.Close, length=self.ema_fast)
        self.ema_slow_series = self.I(ta.ema, self.data.Close, length=self.ema_slow)
        self.volume_ma = self.I(ta.sma, self.data.Volume, length=self.volume_ma_len)

    def next(self):
        if len(self.data.Close) < max(self.bias_window, self.atr_length, self.ema_slow):
            return

        current_close = self.data.Close[-1]
        current_open = self.data.Open[-1]
        current_high = self.data.High[-1]
        current_low = self.data.Low[-1]
        current_volume = self.data.Volume[-1]

        # Directional Bias with magnitude
        up_strength = sum((self.data.Close[-i] - self.data.Open[-i]) for i in range(1, self.bias_window + 1) if self.data.Close[-i] > self.data.Open[-i])
        down_strength = sum((self.data.Open[-i] - self.data.Close[-i]) for i in range(1, self.bias_window + 1) if self.data.Close[-i] < self.data.Open[-i])
        total_strength = up_strength + down_strength if up_strength + down_strength != 0 else 1
        bias_ratio = up_strength / total_strength

        # Range Check
        high_range = max(self.data.High[-self.bias_window:])
        low_range = min(self.data.Low[-self.bias_window:])
        range_perc = (high_range - low_range) / low_range

        # Filters
        volatility_ok = self.atr[-1] > pd.Series(self.atr).rolling(20).mean().iloc[-1] * 0.8
        volume_ok = current_volume > self.volume_ma[-1] * 1.2
        trend_up = self.ema_fast_series[-1] > self.ema_slow_series[-1]
        trend_down = self.ema_fast_series[-1] < self.ema_slow_series[-1]
        adx_ok = self.adx['ADX_14'][-1] > self.adx_threshold

        has_bias_long = bias_ratio >= self.bias_threshold and range_perc > self.range_min and trend_up and adx_ok
        has_bias_short = bias_ratio <= (1 - self.bias_threshold) and range_perc > self.range_min and trend_down and adx_ok

        # Entry Conditions
        if has_bias_long and volume_ok and not self.position:
            self.buy()

        elif has_bias_short and volume_ok and not self.position:
            self.sell()

        # Exit Conditions
        if self.position:
            entry_price = self.position.entry_price
            entry_bar = self.position.entry_bar
            holding_period = len(self.data.Close) - entry_bar
            atr_val = self.atr[-1]

            if self.position.is_long:
                sl = entry_price - atr_val
                tp = entry_price + atr_val * self.risk_reward
                if current_close <= sl or current_close >= tp or holding_period >= self.max_bars:
                    self.position.close()

            elif self.position.is_short:
                sl = entry_price + atr_val
                tp = entry_price - atr_val * self.risk_reward
                if current_close >= sl or current_close <= tp or holding_period >= self.max_bars:
                    self.position.close()



# 6. Price Statistical Strategy

In [ ]:

import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting import Strategy

class Price_Statistical_Strategy(Strategy):
    z_base_length = 3
    short_smooth = 3
    long_smooth = 5
    gap_bars = 5

    def init(self):
        # Calculate rolling mean and std
        self.mean = self.I(lambda x: pd.Series(x).rolling(self.z_base_length).mean(), self.data.Close)
        self.std = self.I(lambda x: pd.Series(x).rolling(self.z_base_length).std(), self.data.Close)
        self.z_raw = (self.data.Close - self.mean) / self.std

        # Smooth the Z-Score
        self.z_short = self.I(lambda z: pd.Series(z).rolling(self.short_smooth).mean(), self.z_raw)
        self.z_long = self.I(lambda z: pd.Series(z).rolling(self.long_smooth).mean(), self.z_raw)

        # Bar index tracker for signal gap logic
        self.last_entry_bar = -np.inf
        self.last_exit_bar = -np.inf

    def is_bullish_3bars(self):
        return (
            self.data.Close[-1] > self.data.Close[-2] and
            self.data.Close[-2] > self.data.Close[-3] and
            self.data.Close[-3] > self.data.Close[-4] and
            self.data.Close[-4] > self.data.Close[-5]
        )

    def is_bearish_3bars(self):
        return (
            self.data.Close[-1] < self.data.Close[-2] and
            self.data.Close[-2] < self.data.Close[-3] and
            self.data.Close[-3] < self.data.Close[-4] and
            self.data.Close[-4] < self.data.Close[-5]
        )

    def next(self):
        if len(self.z_short) < max(self.long_smooth, self.z_base_length) + 5:
            return

        bar = len(self.data.Close) - 1
        z_short = self.z_short[-1]
        z_long = self.z_long[-1]

        # Entry: z_short > z_long with conditions
        if not self.position:
            if (
                z_short > z_long and
                (bar - self.last_entry_bar) > self.gap_bars and
                not self.is_bullish_3bars()
            ):
                self.buy()
                self.last_entry_bar = bar

        # Exit: z_short < z_long with conditions
        if self.position:
            if (
                z_short < z_long and
                (bar - self.last_exit_bar) > self.gap_bars and
                not self.is_bearish_3bars()
            ):
                self.position.close()
                self.last_exit_bar = bar


# Improved Price Statistical Strategy

In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from backtesting import Strategy

class ImprovedPriceStatisticalStrategy(Strategy):
    z_base_length = 3
    short_smooth = 3
    long_smooth = 5
    gap_bars = 5
    atr_length = 14
    volume_sma_len = 20
    ema_trend_len = 200
    risk_reward = 2.0
    max_holding_bars = 20

    def init(self):
        # Indicators
        self.mean = self.I(lambda x: pd.Series(x).rolling(self.z_base_length).mean(), self.data.Close)
        self.std = self.I(lambda x: pd.Series(x).rolling(self.z_base_length).std(), self.data.Close)
        self.z_raw = (self.data.Close - self.mean) / self.std
        self.z_short = self.I(lambda z: pd.Series(z).ewm(span=self.short_smooth).mean(), self.z_raw)
        self.z_long = self.I(lambda z: pd.Series(z).ewm(span=self.long_smooth).mean(), self.z_raw)
        self.atr = self.I(ta.atr, self.data.High, self.data.Low, self.data.Close, length=self.atr_length)
        self.ema200 = self.I(ta.ema, self.data.Close, length=self.ema_trend_len)
        self.vol_ma = self.I(ta.sma, self.data.Volume, length=self.volume_sma_len)

        # Bar trackers
        self.last_entry_bar = -np.inf
        self.last_exit_bar = -np.inf

    def is_bullish_3bars(self):
        return all(self.data.Close[-i] > self.data.Close[-i - 1] for i in range(1, 5))

    def is_bearish_3bars(self):
        return all(self.data.Close[-i] < self.data.Close[-i - 1] for i in range(1, 5))

    def next(self):
        if len(self.data.Close) < max(self.ema_trend_len, self.volume_sma_len, self.z_base_length + 5):
            return

        bar = len(self.data.Close) - 1
        price = self.data.Close[-1]
        z_s = self.z_short[-1]
        z_l = self.z_long[-1]
        atr_val = self.atr[-1]
        volume = self.data.Volume[-1]
        ema_trend = self.ema200[-1]
        vol_filter = volume > self.vol_ma[-1]

        # === ENTRY ===
        if not self.position:
            # Long Entry
            if (
                z_s > z_l and price > ema_trend and vol_filter and not self.is_bullish_3bars()
                and (bar - self.last_entry_bar) > self.gap_bars
            ):
                self.buy()
                self.last_entry_bar = bar

            # Short Entry
            elif (
                z_s < z_l and price < ema_trend and vol_filter and not self.is_bearish_3bars()
                and (bar - self.last_entry_bar) > self.gap_bars
            ):
                self.sell()
                self.last_entry_bar = bar

        # === EXIT ===
        if self.position:
            entry_price = self.position.entry_price
            entry_bar = self.position.entry_bar
            holding_period = len(self.data.Close) - entry_bar

            # Long exit
            if self.position.is_long:
                sl = entry_price - atr_val
                tp = entry_price + atr_val * self.risk_reward
                if price <= sl or price >= tp or z_s < z_l or holding_period >= self.max_holding_bars:
                    self.position.close()
                    self.last_exit_bar = bar

            # Short exit
            elif self.position.is_short:
                sl = entry_price + atr_val
                tp = entry_price - atr_val * self.risk_reward
                if price >= sl or price <= tp or z_s > z_l or holding_period >= self.max_holding_bars:
                    self.position.close()
                    self.last_exit_bar = bar


# 7. Long/Short/Exit/Risk management Strategy

In [ ]:
import pandas as pd
import pandas_ta
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

class PineScriptStrategy(Strategy):
    # General Settings
    tradingDirection = "All"  # Options: "Long Only", "Short Only", "All"
    maxTradesPerDay = 3
    barsToWait = 10

    # Session Settings
    x_cona_sess_use = True
    x_cona_sess = "0930-1300"  # Format: "HHMM-HHMM"
    x_cona_tz = None  # Leave blank for exchange time zone, otherwise specify like "America/New_York"
    x_cona_expr_use = False
    x_cona_expr_timestamp = pd.Timestamp('2024-01-01') # Placeholder, will be converted to datetime

    # Contract Settings (for P&L calculation, not directly used by backtesting.py core for trading)
    # These settings are flexible and can be adapted for any instrument.
    contractType = "Other" # Changed default to 'Other'
    pointValue = 1.0  # Default point value for general stocks (e.g., 1.0 for a typical stock)
    tickSize = 0.01   # Default tick size for general stocks (e.g., 0.01 for most stocks)

    # Risk Management
    riskType = "Percentage"  # Options: "Percentage", "ATR", "Points"
    stopLoss = 2.0           # Stop Loss % (if riskType="Percentage")
    takeProfit = 3.0         # Take Profit % (if riskType="Percentage")
    stopLossPoints = 10.0    # Stop Loss (Points) (if riskType="Points")
    takeProfitPoints = 15.0  # Take Profit (Points) (if riskType="Points")
    useTimeExit = False
    exitBars = 10

    # ATR Settings (if riskType="ATR")
    atrPeriod = 14
    atrMultiplierSL = 2.0
    atrMultiplierTP = 3.0

    # Trailing Stop Settings
    useTrailingStop = False
    trailPoints = 5.0
    activateTrailAtPoints = 10.0

    # Break Even Settings
    useBreakEven = False
    breakEvenAtPoints = 10.0
    breakEvenOffset = 0.1

    # Partial Profit Taking
    usePartialTP = False
    partialTP1Points = 5.0
    partialTP1Size = 50.0  # % of initial position
    partialTP2Points = 10.0
    partialTP2Size = 25.0  # % of initial remaining position

    # Streak Cutoff Settings
    useStreakCutoff = False
    maxWinStreak = 3
    maxLossStreak = 3
    streakLookback = 10
    streakResetDaily = True

    # Trading Conditions
    longSourceField = "Close" # Options: "Open", "High", "Low", "Close"
    longValue = 0.0
    shortSourceField = "Close" # Options: "Open", "High", "Low", "Close"
    shortValue = 0.0

    def init(self):
        # Initialize ATR
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atrPeriod)

        # Initialize internal state variables
        self.tradesToday = 0
        self.barsSinceLastTrade = self.barsToWait
        self.entryPrice = None
        self.tradeBarCount = 0
        self.exitType = None
        self.trailStopPrice = None
        self.bestPrice = None
        self.trailingActive = False
        self.partialTP1Hit = False
        self.partialTP2Hit = False
        self.breakEvenSet = False
        self.currentTradePnL = 0.0
        self.initial_position_size = 0 # To track initial size for partial exits

        # Performance tracking (backtesting.py handles most, but custom streaks need manual)
        self.tradeResults = [] # 1 for win, -1 for loss
        self.streakCutoffActive = False

        # Session time parsing
        if self.x_cona_sess_use and self.x_cona_sess:
            try:
                start_time_str, end_time_str = self.x_cona_sess.split('-')
                self.session_start_time = pd.to_datetime(start_time_str, format='%H%M').time()
                self.session_end_time = pd.to_datetime(end_time_str, format='%H%M').time()
            except ValueError:
                self.session_start_time = None
                self.session_end_time = None
                print(f"Warning: Invalid session time format '{self.x_cona_sess}'. Session check disabled.")
        else:
            self.session_start_time = None
            self.session_end_time = None

        # Determine contract point value
        # Removed specific futures contract values, relying on the 'pointValue' parameter.
        # This makes the strategy more generic for stocks where a 'point' is typically 1.0 (for price per share).
        self.contractPointValue = self.pointValue

    def _is_within_session(self):
        if not self.x_cona_sess_use or self.session_start_time is None:
            return True
        current_time = self.data.index[-1].time()
        # Handle sessions spanning midnight
        if self.session_start_time < self.session_end_time:
            return self.session_start_time <= current_time <= self.session_end_time
        else: # Session crosses midnight
            return current_time >= self.session_start_time or current_time <= self.session_end_time

    def next(self):
        current_close = self.data.Close[-1]
        current_open = self.data.Open[-1]
        current_high = self.data.High[-1]
        current_low = self.data.Low[-1]
        current_date = self.data.index[-1].date()
        previous_date = self.data.index[-2].date() if len(self.data.index) > 1 else None

        # Reset trade counter and streaks at day start
        if previous_date and current_date != previous_date:
            self.tradesToday = 0
            if self.streakResetDaily:
                self.tradeResults = []
                self.streakCutoffActive = False

        # Update bars since last trade counter
        if not self.position:
            self.barsSinceLastTrade += 1

        # Check if trading is allowed
        streakAllowsTrading = not self.useStreakCutoff or not self.streakCutoffActive
        canTrade = (self.tradesToday < self.maxTradesPerDay and
                    self.barsSinceLastTrade >= self.barsToWait and
                    self._is_within_session() and
                    streakAllowsTrading)

        # Get the source values for conditions
        long_source_value = getattr(self.data, self.longSourceField, self.data.Close)[-1]
        short_source_value = getattr(self.data, self.shortSourceField, self.data.Close)[-1]

        # Calculate conditions
        longCondition = long_source_value == self.longValue
        shortCondition = short_source_value == self.shortValue

        # Trade Entry Logic
        if not self.position:
            if canTrade:
                if longCondition and self.tradingDirection != "Short Only":
                    self.buy()
                    self.entryPrice = current_open # Entry happens on the next bar's open in Pine Script if process_orders_on_close=false
                    self.tradeBarCount = 0
                    self.exitType = None
                    self.bestPrice = current_high
                    self.trailingActive = False
                    self.partialTP1Hit = False
                    self.partialTP2Hit = False
                    self.breakEvenSet = False
                    self.initial_position_size = self.position.size
                    self.barsSinceLastTrade = 0
                    self.tradesToday += 1

                elif shortCondition and self.tradingDirection != "Long Only":
                    self.sell()
                    self.entryPrice = current_open # Entry happens on the next bar's open
                    self.tradeBarCount = 0
                    self.exitType = None
                    self.bestPrice = current_low
                    self.trailingActive = False
                    self.partialTP1Hit = False
                    self.partialTP2Hit = False
                    self.breakEvenSet = False
                    self.initial_position_size = self.position.size
                    self.barsSinceLastTrade = 0
                    self.tradesToday += 1
        else: # Position is active
            self.tradeBarCount += 1

            # Calculate SL/TP levels
            stopLevel = 0.0
            profitLevel = 0.0
            current_atr = self.atr[-1]

            if self.position.is_long:
                if self.riskType == "Percentage":
                    stopLevel = self.entryPrice * (1 - self.stopLoss/100)
                    profitLevel = self.entryPrice * (1 + self.takeProfit/100)
                elif self.riskType == "ATR":
                    stopLevel = self.entryPrice - (current_atr * self.atrMultiplierSL)
                    profitLevel = self.entryPrice + (current_atr * self.atrMultiplierTP)
                else: # Points
                    stopLevel = self.entryPrice - self.stopLossPoints
                    profitLevel = self.entryPrice + self.takeProfitPoints
            else: # Short position
                if self.riskType == "Percentage":
                    stopLevel = self.entryPrice * (1 + self.stopLoss/100)
                    profitLevel = self.entryPrice * (1 - self.takeProfit/100)
                elif self.riskType == "ATR":
                    stopLevel = self.entryPrice + (current_atr * self.atrMultiplierSL)
                    profitLevel = self.entryPrice - (current_atr * self.atrMultiplierTP)
                else: # Points
                    stopLevel = self.entryPrice + self.stopLossPoints
                    profitLevel = self.entryPrice - self.takeProfitPoints

            # Update best price achieved
            if self.position.is_long:
                if self.bestPrice is None:
                    self.bestPrice = current_high
                else:
                    self.bestPrice = max(self.bestPrice, current_high)
            else: # Short position
                if self.bestPrice is None:
                    self.bestPrice = current_low
                else:
                    self.bestPrice = min(self.bestPrice, current_low)

            # Calculate current profit in points
            currentProfitPoints = 0.0
            if self.position.is_long:
                currentProfitPoints = current_close - self.entryPrice
            else:
                currentProfitPoints = self.entryPrice - current_close

            # Move stop to break-even
            if self.useBreakEven and not self.breakEvenSet and currentProfitPoints >= self.breakEvenAtPoints:
                self.breakEvenSet = True
                if self.position.is_long:
                    stopLevel = self.entryPrice + self.breakEvenOffset
                else:
                    stopLevel = self.entryPrice - self.breakEvenOffset

            # Activate trailing stop
            if self.useTrailingStop and not self.trailingActive and currentProfitPoints >= self.activateTrailAtPoints:
                self.trailingActive = True
                if self.position.is_long:
                    self.trailStopPrice = current_high - self.trailPoints
                else:
                    self.trailStopPrice = current_low + self.trailPoints

            # Update trailing stop if active
            if self.useTrailingStop and self.trailingActive:
                if self.position.is_long:
                    newTrailStop = current_high - self.trailPoints
                    self.trailStopPrice = max(self.trailStopPrice, newTrailStop)
                    if not self.breakEvenSet or self.trailStopPrice > stopLevel:
                        stopLevel = self.trailStopPrice
                else: # Short position
                    newTrailStop = current_low + self.trailPoints
                    self.trailStopPrice = min(self.trailStopPrice, newTrailStop)
                    if not self.breakEvenSet or self.trailStopPrice < stopLevel:
                        stopLevel = self.trailStopPrice

            # Partial Profit Taking
            if self.usePartialTP:
                if self.position.is_long:
                    partialTP1Price = self.entryPrice + self.partialTP1Points
                    partialTP2Price = self.entryPrice + self.partialTP2Points
                    # Check for partial TP1
                    if not self.partialTP1Hit and current_high >= partialTP1Price:
                        self.close(size=self.initial_position_size * (self.partialTP1Size / 100))
                        self.partialTP1Hit = True
                    # Check for partial TP2
                    if self.partialTP1Hit and not self.partialTP2Hit and current_high >= partialTP2Price:
                        self.close(size=self.initial_position_size * (self.partialTP2Size / 100))
                        self.partialTP2Hit = True
                else: # Short position
                    partialTP1Price = self.entryPrice - self.partialTP1Points
                    partialTP2Price = self.entryPrice - self.partialTP2Points
                    # Check for partial TP1
                    if not self.partialTP1Hit and current_low <= partialTP1Price:
                        self.close(size=self.initial_position_size * (self.partialTP1Size / 100))
                        self.partialTP1Hit = True
                    # Check for partial TP2
                    if self.partialTP1Hit and not self.partialTP2Hit and current_low <= partialTP2Price:
                        self.close(size=self.initial_position_size * (self.partialTP2Size / 100))
                        self.partialTP2Hit = True

            # Set the stop loss and take profit for the position
            # backtesting.py allows dynamic SL/TP adjustments
            if self.position.is_long:
                self.position.sl = stopLevel
                self.position.tp = profitLevel
            else: # Short position
                self.position.sl = stopLevel
                self.position.tp = profitLevel

            # Time exit condition (if no SL/TP hit)
            if self.useTimeExit and self.tradeBarCount >= self.exitBars:
                self.position.close()
                self.exitType = "TIME EXIT"

        # Update streak tracking variables when a trade closes
        if self.position.is_closed and self.entryPrice is not None:
            trade_profit_loss = (self.data.Close[-1] - self.entryPrice) * (1 if self.position.is_long else -1)
            trade_pnl_dollars = trade_profit_loss * self.contractPointValue

            if self.useStreakCutoff:
                if trade_pnl_dollars > 0:
                    self.tradeResults.insert(0, 1) # Win
                else:
                    self.tradeResults.insert(0, -1) # Loss

                if len(self.tradeResults) > self.streakLookback:
                    self.tradeResults.pop()

                winCount = self.tradeResults.count(1)
                lossCount = self.tradeResults.count(-1)
                self.streakCutoffActive = (winCount >= self.maxWinStreak) or (lossCount >= self.maxLossStreak)

            # Reset variables after trade closure
            self.entryPrice = None
            self.exitType = None
            self.tradeBarCount = 0
            self.bestPrice = None
            self.trailingActive = False
            self.partialTP1Hit = False
            self.partialTP2Hit = False
            self.breakEvenSet = False
            self.currentTradePnL = 0.0
            self.initial_position_size = 0

# Improved Long/Short/Exit/Risk management Strategy

In [ ]:
import pandas as pd
import pandas_ta
from backtesting import Strategy, Backtest
from backtesting.lib import crossover, resample_apply

class ImprovedRsiDivergenceStrategy(Strategy):
    # ##################
    # General Settings
    # ##################
    # Determines if the strategy takes "Long Only", "Short Only", or "All" types of trades.
    tradingDirection = "All"
    # Maximum number of trades allowed per trading day.
    maxTradesPerDay = 3
    # Minimum number of bars to wait after a trade closes before entering another.
    barsToWait = 10

    # ##################
    # Session Settings
    # ##################
    # Enable/disable session filtering for trades.
    x_cona_sess_use = True
    # Define the trading session in "HHMM-HHMM" format (e.g., "0930-1300").
    x_cona_sess = "0930-1300"
    # Time zone for the session. Leave None for exchange time zone. Example: "America/New_York"
    x_cona_tz = None
    # Enable/disable an expiration timestamp for the strategy.
    x_cona_expr_use = False
    # The expiration timestamp. Trades will not be taken after this time.
    x_cona_expr_timestamp = pd.Timestamp('2024-01-01') # Default placeholder

    # ##################
    # Contract/Instrument Settings
    # These settings are crucial for accurate P&L calculation within the strategy
    # and are generalized for all stock types (e.g., Nifty 50, Nifty 250, individual stocks).
    # ##################
    # A descriptive type for the instrument being traded.
    contractType = "Other"
    # Dollar value per "point" of price movement.
    # For most stocks, this is typically 1.0 (since 1 point = $1 per share).
    pointValue = 1.0
    # Minimum price movement (tick size). For most stocks, this is 0.01.
    tickSize = 0.01

    # ##################
    # Risk Management
    # ##################
    # Type of stop loss and take profit calculation: "Percentage", "ATR", or "Points".
    riskType = "Percentage"
    # Stop Loss percentage of entry price (if riskType="Percentage").
    stopLoss = 2.0
    # Take Profit percentage of entry price (if riskType="Percentage").
    takeProfit = 3.0
    # Stop Loss in points (if riskType="Points").
    stopLossPoints = 10.0
    # Take Profit in points (if riskType="Points").
    takeProfitPoints = 15.0
    # Enable/disable a time-based exit as a backup.
    useTimeExit = False
    # Number of bars after which to exit if no SL/TP is hit (if useTimeExit is true).
    exitBars = 10

    # ATR Settings (only applicable if riskType="ATR")
    # Period for Average True Range (ATR) calculation.
    atrPeriod = 14
    # Multiplier for ATR to determine Stop Loss level.
    atrMultiplierSL = 2.0
    # Multiplier for ATR to determine Take Profit level.
    atrMultiplierTP = 3.0

    # Trailing Stop Settings
    # Enable/disable trailing stop.
    useTrailingStop = False
    # Trailing stop distance in points from the highest (for long) or lowest (for short) price reached.
    trailPoints = 5.0
    # Profit in points at which the trailing stop becomes active.
    activateTrailAtPoints = 10.0

    # Break Even Settings
    # Enable/disable moving stop loss to break even.
    useBreakEven = False
    # Profit in points at which to move stop loss to break even.
    breakEvenAtPoints = 10.0
    # Small offset in points beyond entry price for break-even stop loss (to cover commissions/slippage).
    breakEvenOffset = 0.1

    # Partial Profit Taking
    # Enable/disable taking partial profits.
    usePartialTP = False
    # Points distance from entry for the first partial take profit.
    partialTP1Points = 5.0
    # Percentage of the initial position to close at the first partial TP.
    partialTP1Size = 50.0
    # Points distance from entry for the second partial take profit.
    partialTP2Points = 10.0
    # Percentage of the initial position to close at the second partial TP.
    partialTP2Size = 25.0

    # Streak Cutoff Settings
    # Enable/disable stopping trading after a certain number of consecutive wins or losses.
    useStreakCutoff = False
    # Maximum number of consecutive winning trades before stopping.
    maxWinStreak = 3
    # Maximum number of consecutive losing trades before stopping.
    maxLossStreak = 3
    # Number of recent closed trades to consider for streak counting.
    streakLookback = 10
    # Reset win/loss streak counters at the start of each new day.
    streakResetDaily = True

    # ##################
    # Improved Trading Conditions (RSI for Overbought/Oversold & SMA for Trend)
    # ##################
    # Period for Relative Strength Index (RSI) calculation.
    rsiPeriod = 14
    # RSI level considered oversold, signaling a potential long entry.
    rsiOversold = 30
    # RSI level considered overbought, signaling a potential short entry.
    rsiOverbought = 70
    # Period for Simple Moving Average (SMA) used as a trend filter.
    smaPeriod = 200

    # ##################
    # Position Sizing
    # ##################
    # Percentage of total equity to risk on a single trade.
    # This determines the number of shares/contracts to buy/sell.
    riskPerTradePercent = 1.0 # e.g., 1.0 means risk 1% of equity per trade.

    def init(self):
        """
        Initializes the strategy, setting up indicators and internal state variables.
        This method is called once at the beginning of the backtest.
        """
        # Calculate Average True Range (ATR) using TA-Lib.
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atrPeriod)

        # Calculate Relative Strength Index (RSI) using TA-Lib.
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsiPeriod)

        # Calculate Simple Moving Average (SMA) for trend filtering.
        self.sma = self.I(talib.SMA, self.data.Close, self.smaPeriod)

        # Initialize internal state variables for trade tracking and risk management.
        self.tradesToday = 0
        self.barsSinceLastTrade = self.barsToWait
        self.entryPrice = None          # Stores the price at which the current trade was entered.
        self.tradeBarCount = 0          # Counts the number of bars the current trade has been active.
        self.exitType = None            # Stores the type of exit (e.g., SL, TP, Time Exit).
        self.trailStopPrice = None      # Current level of the trailing stop.
        self.bestPrice = None           # Tracks the most favorable price reached during a trade.
        self.trailingActive = False     # Flag to indicate if trailing stop is active.
        self.partialTP1Hit = False      # Flag for first partial take profit.
        self.partialTP2Hit = False      # Flag for second partial take profit.
        self.breakEvenSet = False       # Flag to indicate if stop has been moved to break-even.
        self.currentTradePnL = 0.0      # Not directly used by backtesting.py for its reports, but for custom logic.
        self.initial_position_size = 0  # Stores the initial size of a position for partial profit calculations.

        # Variables for custom streak tracking.
        self.tradeResults = []          # List to store results of recent trades (1 for win, -1 for loss).
        self.streakCutoffActive = False # Flag to indicate if trading is halted due to streak cutoff.

        # Parse and store session start/end times if session filtering is enabled.
        if self.x_cona_sess_use and self.x_cona_sess:
            try:
                start_time_str, end_time_str = self.x_cona_sess.split('-')
                self.session_start_time = pd.to_datetime(start_time_str, format='%H%M').time()
                self.session_end_time = pd.to_datetime(end_time_str, format='%H%MM').time()
            except ValueError:
                # Handle cases where session time format is invalid.
                self.session_start_time = None
                self.session_end_time = None
                print(f"Warning: Invalid session time format '{self.x_cona_sess}'. Session check disabled.")
        else:
            self.session_start_time = None
            self.session_end_time = None

        # Set the contract point value. For most stocks, one "point" (e.g., $1 price change) equals $1 value.
        self.contractPointValue = self.pointValue

    def _is_within_session(self):
        """
        Checks if the current bar's time falls within the defined trading session.
        Handles sessions that cross midnight.
        """
        if not self.x_cona_sess_use or self.session_start_time is None:
            return True # Session filtering is disabled or invalid, so always within session.

        current_time = self.data.index[-1].time()

        if self.session_start_time < self.session_end_time:
            # Standard session within a single day (e.g., 09:30 - 13:00)
            return self.session_start_time <= current_time <= self.session_end_time
        else:
            # Session crosses midnight (e.g., 22:00 - 04:00)
            return current_time >= self.session_start_time or current_time <= self.session_end_time

    def _calculate_initial_stop_distance(self, current_price, is_long):
        """
        Calculates the initial stop loss distance in points based on the chosen risk type.
        This is crucial for position sizing before entering a trade.
        """
        if self.riskType == "Percentage":
            # For a long, SL is below entry, so distance is percentage * entryPrice
            # For a short, SL is above entry, distance is percentage * entryPrice
            return current_price * (self.stopLoss / 100)
        elif self.riskType == "ATR":
            # ATR is a points value, so directly use ATR * multiplier
            return self.atr[-1] * self.atrMultiplierSL
        else: # riskType == "Points"
            # Direct points value from input
            return self.stopLossPoints

    def next(self):
        """
        This method is called on each new bar of the historical data.
        It contains the core trading logic for entries, exits, and risk management.
        """
        # Get current bar's OHLC data for calculations.
        current_close = self.data.Close[-1]
        current_open = self.data.Open[-1]
        current_high = self.data.High[-1]
        current_low = self.data.Low[-1]
        current_date = self.data.index[-1].date()

        # Get the previous bar's date to detect day changes.
        previous_date = self.data.index[-2].date() if len(self.data.index) > 1 else None

        # Reset daily trade counter and streak counters at the start of a new day.
        if previous_date and current_date != previous_date:
            self.tradesToday = 0
            if self.streakResetDaily:
                self.tradeResults = [] # Clear the streak history
                self.streakCutoffActive = False # Deactivate cutoff for the new day

        # Increment counter for bars since the last trade, when no position is open.
        if not self.position:
            self.barsSinceLastTrade += 1

        # Check if trading is currently allowed based on various constraints.
        streakAllowsTrading = not self.useStreakCutoff or not self.streakCutoffActive
        canTrade = (self.tradesToday < self.maxTradesPerDay and
                    self.barsSinceLastTrade >= self.barsToWait and
                    self._is_within_session() and
                    streakAllowsTrading)

        # Get current indicator values.
        current_rsi = self.rsi[-1]
        current_sma = self.sma[-1]

        # ##################
        # Improved Trade Entry Conditions (RSI + SMA Trend Filter)
        # ##################
        # Long Condition: Price is above SMA (uptrend) AND RSI is oversold.
        longCondition = (current_close > current_sma) and (current_rsi < self.rsiOversold)
        # Short Condition: Price is below SMA (downtrend) AND RSI is overbought.
        shortCondition = (current_close < current_sma) and (current_rsi > self.rsiOverbought)

        # ##################
        # Trade Entry Logic
        # ##################
        if not self.position: # Check if there is no open position
            if canTrade: # Check if general trading conditions are met

                # Calculate trade size based on risk per trade and initial stop distance.
                # We use current_open for the hypothetical entry price to calculate initial stop for sizing.
                potential_entry_price = current_open # Assuming entry at next bar's open.
                num_shares = 0

                # Attempt to enter a Long trade
                if longCondition and self.tradingDirection != "Short Only":
                    # Calculate initial stop loss distance for sizing
                    initial_sl_distance_points = self._calculate_initial_stop_distance(potential_entry_price, is_long=True)
                    if initial_sl_distance_points > 0: # Avoid division by zero
                        risk_per_share_dollar = initial_sl_distance_points * self.contractPointValue
                        dollars_to_risk = self.equity * (self.riskPerTradePercent / 100)
                        num_shares = int(dollars_to_risk / risk_per_share_dollar)
                        num_shares = max(1, num_shares) # Ensure at least 1 share

                    if num_shares > 0: # Only place order if a valid number of shares is determined
                        self.buy(size=num_shares) # Execute a buy order with calculated size
                        self.entryPrice = potential_entry_price
                        self.tradeBarCount = 0
                        self.exitType = None
                        self.bestPrice = current_high # Initialize best price for trailing stop.
                        self.trailingActive = False
                        self.partialTP1Hit = False
                        self.partialTP2Hit = False
                        self.breakEvenSet = False
                        self.initial_position_size = self.position.size # Store initial size for partial exits.
                        self.barsSinceLastTrade = 0 # Reset bars since last trade counter.
                        self.tradesToday += 1 # Increment daily trade counter.

                # Attempt to enter a Short trade
                elif shortCondition and self.tradingDirection != "Long Only":
                    # Calculate initial stop loss distance for sizing
                    initial_sl_distance_points = self._calculate_initial_stop_distance(potential_entry_price, is_long=False)
                    if initial_sl_distance_points > 0: # Avoid division by zero
                        risk_per_share_dollar = initial_sl_distance_points * self.contractPointValue
                        dollars_to_risk = self.equity * (self.riskPerTradePercent / 100)
                        num_shares = int(dollars_to_risk / risk_per_share_dollar)
                        num_shares = max(1, num_shares) # Ensure at least 1 share

                    if num_shares > 0: # Only place order if a valid number of shares is determined
                        self.sell(size=num_shares) # Execute a sell (short) order with calculated size
                        self.entryPrice = potential_entry_price
                        self.tradeBarCount = 0
                        self.exitType = None
                        self.bestPrice = current_low # Initialize best price for trailing stop.
                        self.trailingActive = False
                        self.partialTP1Hit = False
                        self.partialTP2Hit = False
                        self.breakEvenSet = False
                        self.initial_position_size = self.position.size
                        self.barsSinceLastTrade = 0
                        self.tradesToday += 1
        else: # A position is currently active
            self.tradeBarCount += 1 # Increment bar count for the active trade.

            # Calculate Stop Loss (SL) and Take Profit (TP) levels based on chosen riskType.
            stopLevel = 0.0
            profitLevel = 0.0
            current_atr = self.atr[-1] # Get the current ATR value.

            if self.position.is_long:
                if self.riskType == "Percentage":
                    stopLevel = self.entryPrice * (1 - self.stopLoss/100)
                    profitLevel = self.entryPrice * (1 + self.takeProfit/100)
                elif self.riskType == "ATR":
                    stopLevel = self.entryPrice - (current_atr * self.atrMultiplierSL)
                    profitLevel = self.entryPrice + (current_atr * self.atrMultiplierTP)
                else: # riskType == "Points"
                    stopLevel = self.entryPrice - self.stopLossPoints
                    profitLevel = self.entryPrice + self.takeProfitPoints
            else: # Short position
                if self.riskType == "Percentage":
                    stopLevel = self.entryPrice * (1 + self.stopLoss/100)
                    profitLevel = self.entryPrice * (1 - self.takeProfit/100)
                elif self.riskType == "ATR":
                    stopLevel = self.entryPrice + (current_atr * self.atrMultiplierSL)
                    profitLevel = self.entryPrice - (current_atr * self.atrMultiplierTP)
                else: # riskType == "Points"
                    stopLevel = self.entryPrice + self.stopLossPoints
                    profitLevel = self.entryPrice - self.takeProfitPoints

            # Update the 'bestPrice' achieved so far within the current trade.
            if self.position.is_long:
                if self.bestPrice is None:
                    self.bestPrice = current_high
                else:
                    self.bestPrice = max(self.bestPrice, current_high)
            else: # Short position
                if self.bestPrice is None:
                    self.bestPrice = current_low
                else:
                    self.bestPrice = min(self.bestPrice, current_low)

            # Calculate current profit in points.
            currentProfitPoints = 0.0
            if self.position.is_long:
                currentProfitPoints = current_close - self.entryPrice
            else:
                currentProfitPoints = self.entryPrice - current_close

            # ##################
            # Break-Even Logic
            # ##################
            if self.useBreakEven and not self.breakEvenSet and currentProfitPoints >= self.breakEvenAtPoints:
                self.breakEvenSet = True
                if self.position.is_long:
                    stopLevel = self.entryPrice + self.breakEvenOffset
                else:
                    stopLevel = self.entryPrice - self.breakEvenOffset

            # ##################
            # Trailing Stop Logic
            # ##################
            # Activate trailing stop when profit threshold is met.
            if self.useTrailingStop and not self.trailingActive and currentProfitPoints >= self.activateTrailAtPoints:
                self.trailingActive = True
                if self.position.is_long:
                    self.trailStopPrice = current_high - self.trailPoints
                else:
                    self.trailStopPrice = current_low + self.trailPoints

            # Update trailing stop level if active.
            if self.useTrailingStop and self.trailingActive:
                if self.position.is_long:
                    newTrailStop = current_high - self.trailPoints
                    self.trailStopPrice = max(self.trailStopPrice, newTrailStop)
                    # If trailing stop is higher than current SL (or break-even), update SL.
                    if not self.breakEvenSet or self.trailStopPrice > stopLevel:
                        stopLevel = self.trailStopPrice
                else: # Short position
                    newTrailStop = current_low + self.trailPoints
                    self.trailStopPrice = min(self.trailStopPrice, newTrailStop)
                    # If trailing stop is lower than current SL (or break-even), update SL.
                    if not self.breakEvenSet or self.trailStopPrice < stopLevel:
                        stopLevel = self.trailStopPrice

            # ##################
            # Partial Profit Taking Logic
            # ##################
            if self.usePartialTP:
                if self.position.is_long:
                    partialTP1Price = self.entryPrice + self.partialTP1Points
                    partialTP2Price = self.entryPrice + self.partialTP2Points
                    # Check for first partial TP
                    if not self.partialTP1Hit and current_high >= partialTP1Price:
                        # Close a percentage of the initial position size.
                        # Ensure we don't try to close more than current position size.
                        size_to_close = self.initial_position_size * (self.partialTP1Size / 100)
                        self.close(size=min(size_to_close, self.position.size))
                        self.partialTP1Hit = True
                    # Check for second partial TP (only if first was hit)
                    if self.partialTP1Hit and not self.partialTP2Hit and current_high >= partialTP2Price:
                        size_to_close = self.initial_position_size * (self.partialTP2Size / 100)
                        self.close(size=min(size_to_close, self.position.size))
                        self.partialTP2Hit = True
                else: # Short position
                    partialTP1Price = self.entryPrice - self.partialTP1Points
                    partialTP2Price = self.entryPrice - self.partialTP2Points
                    # Check for first partial TP
                    if not self.partialTP1Hit and current_low <= partialTP1Price:
                        size_to_close = self.initial_position_size * (self.partialTP1Size / 100)
                        self.close(size=min(size_to_close, self.position.size))
                        self.partialTP1Hit = True
                    # Check for second partial TP (only if first was hit)
                    if self.partialTP1Hit and not self.partialTP2Hit and current_low <= partialTP2Price:
                        size_to_close = self.initial_position_size * (self.partialTP2Size / 100)
                        self.close(size=min(size_to_close, self.position.size))
                        self.partialTP2Hit = True

            # ##################
            # Set Dynamic SL/TP for the current position
            # backtesting.py allows updating these on every bar.
            # ##################
            if self.position.is_long:
                self.position.sl = stopLevel
                self.position.tp = profitLevel
            else: # Short position
                self.position.sl = stopLevel
                self.position.tp = profitLevel

            # ##################
            # Time-based Exit Condition
            # ##################
            if self.useTimeExit and self.tradeBarCount >= self.exitBars:
                self.position.close() # Close the entire position.
                self.exitType = "TIME EXIT" # Mark the exit type.

        # ##################
        # Post-Trade Closure Logic (for custom metrics like streak tracking)
        # This block executes when a position has just closed (either by SL, TP, or time exit).
        # Note: For strict P&L, refer to backtesting.py's `_trades` DataFrame after `bt.run()`.
        # This in-`next()` calculation provides an approximation for immediate streak tracking.
        # ##################
        if self.position.is_closed and self.entryPrice is not None:
            # Calculate the hypothetical P&L of the just-closed trade in points.
            trade_profit_loss_points = (self.data.Close[-1] - self.entryPrice) * (1 if self.position.is_long else -1)
            # Convert points P&L to dollar P&L using the defined contractPointValue.
            trade_pnl_dollars = trade_profit_loss_points * self.contractPointValue

            # Update streak tracking if enabled.
            if self.useStreakCutoff:
                if trade_pnl_dollars > 0:
                    self.tradeResults.insert(0, 1) # Add 1 for a winning trade to the beginning of the list.
                else:
                    self.tradeResults.insert(0, -1) # Add -1 for a losing trade.

                # Keep the `tradeResults` list within the specified `streakLookback` size.
                if len(self.tradeResults) > self.streakLookback:
                    self.tradeResults.pop() # Remove the oldest trade result.

                # Count current wins and losses in the lookback window.
                winCount = self.tradeResults.count(1)
                lossCount = self.tradeResults.count(-1)

                # Determine if streak cutoff should be active.
                self.streakCutoffActive = (winCount >= self.maxWinStreak) or (lossCount >= self.maxLossStreak)

            # Reset all trade-specific state variables after a position closes.
            self.entryPrice = None
            self.exitType = None
            self.tradeBarCount = 0
            self.bestPrice = None
            self.trailingActive = False
            self.partialTP1Hit = False
            self.partialTP2Hit = False
            self.breakEvenSet = False
            self.currentTradePnL = 0.0
            self.initial_position_size = 0


# #############################################################################
# Example Usage with backtesting.py
# #############################################################################

# To run this strategy, you need historical OHLCV data in a Pandas DataFrame.
# The DataFrame must have a DatetimeIndex and columns named 'Open', 'High', 'Low', 'Close', 'Volume'.

# Example with dummy data (replace with your actual stock data)
# from datetime import datetime, timedelta
# import numpy as np

# def generate_dummy_data(start_date, periods, interval_minutes=5):
#     dates = [start_date + timedelta(minutes=i*interval_minutes) for i in range(periods)]
#     np.random.seed(42)
#     open_prices = 100 + np.cumsum(np.random.randn(periods))
#     high_prices = open_prices + np.abs(np.random.randn(periods) * 0.5)
#     low_prices = open_prices - np.abs(np.random.randn(periods) * 0.5)
#     close_prices = open_prices + np.random.randn(periods) * 0.2
#     volume = np.random.randint(1000, 5000, periods)

#     df = pd.DataFrame({
#         'Open': open_prices,
#         'High': high_prices,
#         'Low': low_prices,
#         'Close': close_prices,
#         'Volume': volume
#     }, index=pd.to_datetime(dates))
#     return df

# # Example: Generate 1000 bars of 5-minute data
# # data = generate_dummy_data(datetime(2023, 1, 1, 9, 0, 0), 1000, 5)

# # Uncomment the following lines to run a backtest with example data or your own loaded data:
# # bt = Backtest(data, ImprovedRsiDivergenceStrategy,
# #               cash=100_000,          # Starting capital
# #               commission=.001,       # 0.1% commission per trade
# #               exclusive_orders=True) # Only one order (buy/sell) can be open at a time

# # You can customize strategy parameters here:
# # stats = bt.run(
# #     tradingDirection="All",
# #     maxTradesPerDay=5,
# #     barsToWait=5,
# #     riskType="ATR",
# #     atrPeriod=20,
# #     atrMultiplierSL=2.5,
# #     atrMultiplierTP=4.0,
# #     useTrailingStop=True,
# #     trailPoints=0.5,
# #     activateTrailAtPoints=1.0,
# #     useBreakEven=True,
# #     breakEvenAtPoints=0.75,
# #     breakEvenOffset=0.01,
# #     usePartialTP=True,
# #     partialTP1Points=1.0,
# #     partialTP1Size=30.0,
# #     partialTP2Points=2.0,
# #     partialTP2Size=20.0,
# #     useStreakCutoff=True,
# #     maxWinStreak=2,
# #     maxLossStreak=2,
# #     streakLookback=5,
# #     rsiPeriod=14,           # RSI period
# #     rsiOversold=30,         # RSI oversold level for long entry
# #     rsiOverbought=70,       # RSI overbought level for short entry
# #     smaPeriod=50,           # SMA period for trend filter
# #     riskPerTradePercent=1.5 # Risk 1.5% of equity per trade
# # )

# # print(stats)
# # bt.plot()

# 8. Original SigmaStack Strategy

In [ ]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas_ta as ta

class MREBA_DayTradeStrategy(Strategy):
    # ============================================================================
    # STRATEGY PARAMETERS AND INPUTS
    # ============================================================================

    # Risk Management Inputs
    risk_percent = 1.5
    max_daily_loss = 5.0
    max_positions = 3
    reward_ratio = 2.0

    # Technical Indicator Inputs
    macd_fast = 12
    macd_slow = 26
    macd_signal = 9

    rsi_length = 14
    rsi_oversold = 30
    rsi_overbought = 70

    ema_fast = 9
    ema_slow = 21

    atr_length = 14
    atr_multiplier = 2.0
    trail_atr_mult = 1.5

    bb_length = 20
    bb_mult = 2.0

    # Time Filter Inputs (Hardcoded for this example)
    trading_session_start_hour = 9
    trading_session_start_minute = 30
    trading_session_end_hour = 11
    trading_session_end_minute = 30

    close_session_start_hour = 15
    close_session_start_minute = 30

    avoid_lunch = True

    # Volume Filter
    min_volume_mult = 1.2

    # Internal variables
    long_trail_stop = None
    short_trail_stop = None


    def init(self):
        # ============================================================================
        # TECHNICAL INDICATORS
        # ============================================================================

        # MACD
        self.macd = self.I(ta.macd, pd.Series(self.data.Close), fast=self.macd_fast, slow=self.macd_slow, signal=self.macd_signal, plot=True)

        # RSI
        self.rsi = self.I(ta.rsi, pd.Series(self.data.Close), length=self.rsi_length, plot=True)

        # EMAs
        self.ema_9 = self.I(ta.ema, pd.Series(self.data.Close), length=self.ema_fast, plot=True)
        self.ema_21 = self.I(ta.ema, pd.Series(self.data.Close), length=self.ema_slow, plot=True)

        # ATR
        self.atr_value = self.I(ta.atr, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), length=self.atr_length, plot=True)

        # Bollinger Bands
        self.bbands = self.I(ta.bbands, pd.Series(self.data.Close), length=self.bb_length, std=self.bb_mult, plot=True)

        # VWAP
        self.vwap = self.I(ta.vwap, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), pd.Series(self.data.Volume), plot=True)

        # Volume Analysis
        self.volume_sma = self.I(ta.sma, pd.Series(self.data.Volume), length=20, plot=False)

        # ============================================================================
        # MULTI-TIMEFRAME ANALYSIS
        # ============================================================================

        # This is a simplified approach to MTF analysis in backtesting.py
        # For a more robust solution, you might need to preprocess data
        # to align different timeframes correctly.

        # Assuming the base timeframe is 5 minutes or less
        htf_timeframe = '15T'

        # Resample data to the higher timeframe
        htf_data = self.data.df.resample(htf_timeframe).agg({
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum'
        }).dropna()

        htf_ema_fast = ta.ema(htf_data['Close'], length=self.ema_fast)
        htf_ema_slow = ta.ema(htf_data['Close'], length=self.ema_slow)

        self.htf_trend_up = htf_ema_fast > htf_ema_slow
        self.htf_trend_down = htf_ema_fast < htf_ema_slow


    def next(self):
        # Current bar's time
        current_time = self.data.index[-1].time()

        # ============================================================================
        # TIME FILTERS
        # ============================================================================

        in_trading_session = (current_time.hour > self.trading_session_start_hour or \
                             (current_time.hour == self.trading_session_start_hour and current_time.minute >= self.trading_session_start_minute)) and \
                             (current_time.hour < self.trading_session_end_hour or \
                             (current_time.hour == self.trading_session_end_hour and current_time.minute < self.trading_session_end_minute))

        in_close_session = current_time.hour > self.close_session_start_hour or \
                           (current_time.hour == self.close_session_start_hour and current_time.minute >= self.close_session_start_minute)

        lunch_hour = self.avoid_lunch and (current_time.hour == 11 and current_time.minute >= 30) or \
                     (current_time.hour > 11 and current_time.hour < 13) or \
                     (current_time.hour == 13 and current_time.minute < 30)

        time_filter = in_trading_session and not lunch_hour

        # ============================================================================
        # MARKET CONDITIONS AND FILTERS
        # ============================================================================

        atr_sma = ta.sma(self.atr_value, 20)
        volatility_filter = self.atr_value[-1] > atr_sma[-1] * 0.8

        strong_uptrend = self.ema_9[-1] > self.ema_21[-1] and self.data.Close[-1] > self.ema_9[-1] #and self.htf_trend_up.iloc[-1]
        strong_downtrend = self.ema_9[-1] < self.ema_21[-1] and self.data.Close[-1] < self.ema_9[-1] #and self.htf_trend_down.iloc[-1]

        # Volume Filter
        high_volume = self.data.Volume[-1] > self.volume_sma[-1] * self.min_volume_mult

        # ============================================================================
        # ENTRY CONDITIONS
        # ============================================================================

        # Long Entry Conditions
        macd_bullish = crossover(self.macd['MACD_12_26_9'], self.macd['MACDs_12_26_9'])
        rsi_not_overbought = self.rsi[-1] < self.rsi_overbought and self.rsi[-1] > 40
        price_above_vwap = self.data.Close[-1] > self.vwap[-1]
        bullish_momentum = self.data.Close[-1] > self.data.Open[-1] and self.data.High[-1] > self.data.High[-2]

        long_condition = strong_uptrend and macd_bullish and rsi_not_overbought and price_above_vwap and bullish_momentum and high_volume and volatility_filter and time_filter

        # Short Entry Conditions
        macd_bearish = crossover(self.macd['MACDs_12_26_9'], self.macd['MACD_12_26_9'])
        rsi_not_oversold = self.rsi[-1] > self.rsi_oversold and self.rsi[-1] < 60
        price_below_vwap = self.data.Close[-1] < self.vwap[-1]
        bearish_momentum = self.data.Close[-1] < self.data.Open[-1] and self.data.Low[-1] < self.data.Low[-2]

        short_condition = strong_downtrend and macd_bearish and rsi_not_oversold and price_below_vwap and bearish_momentum and high_volume and volatility_filter and time_filter

        # ============================================================================
        # RISK MANAGEMENT
        # ============================================================================

        daily_loss_limit = self.equity * (self.max_daily_loss / 100)
        daily_loss_exceeded = self.equity - self.initial_capital < -daily_loss_limit

        max_positions_reached = len(self.trades) >= self.max_positions

        # ============================================================================
        # STRATEGY ENTRIES
        # ============================================================================

        if long_condition and not self.position and not daily_loss_exceeded and not max_positions_reached:
            long_stop_price = self.data.Close[-1] - (self.atr_value[-1] * self.atr_multiplier)
            long_target = self.data.Close[-1] + (self.data.Close[-1] - long_stop_price) * self.reward_ratio

            # Position sizing
            risk_amount = self.equity * (self.risk_percent / 100)
            position_size = risk_amount / (self.data.Close[-1] - long_stop_price)

            self.buy(sl=long_stop_price, tp=long_target, size=position_size)

        if short_condition and not self.position and not daily_loss_exceeded and not max_positions_reached:
            short_stop_price = self.data.Close[-1] + (self.atr_value[-1] * self.atr_multiplier)
            short_target = self.data.Close[-1] - (short_stop_price - self.data.Close[-1]) * self.reward_ratio

            # Position sizing
            risk_amount = self.equity * (self.risk_percent / 100)
            position_size = risk_amount / (short_stop_price - self.data.Close[-1])

            self.sell(sl=short_stop_price, tp=short_target, size=position_size)

        # ============================================================================
        # TRAILING STOPS
        # ============================================================================

        if self.position.is_long:
            trail_stop_level = self.data.Close[-1] - (self.atr_value[-1] * self.trail_atr_mult)
            if self.long_trail_stop is None or trail_stop_level > self.long_trail_stop:
                self.long_trail_stop = trail_stop_level

            if self.data.Close[-1] <= self.long_trail_stop:
                self.position.close()
                self.long_trail_stop = None

        elif self.position.is_short:
            trail_stop_level = self.data.Close[-1] + (self.atr_value[-1] * self.trail_atr_mult)
            if self.short_trail_stop is None or trail_stop_level < self.short_trail_stop:
                self.short_trail_stop = trail_stop_level

            if self.data.Close[-1] >= self.short_trail_stop:
                self.position.close()
                self.short_trail_stop = None

        # Reset trail stops if position is closed
        if not self.position:
            self.long_trail_stop = None
            self.short_trail_stop = None


        # ============================================================================
        # TIME-BASED EXITS
        # ============================================================================

        if in_close_session and self.position:
            self.position.close()
            self.long_trail_stop = None
            self.short_trail_stop = None



# Improved SigmaStack Strategy

In [ ]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas_ta as ta

class ImprovedMREBA(Strategy):
    # Parameter dictionary for optimization and clarity
    params = dict(
        risk_percent=1.5,
        max_daily_loss=5.0,
        max_positions=3,
        reward_ratio=2.0,
        macd_fast=12,
        macd_slow=26,
        macd_signal=9,
        rsi_length=14,
        rsi_oversold=30,
        rsi_overbought=70,
        ema_fast=9,
        ema_slow=21,
        atr_length=14,
        atr_multiplier=2.0,
        trail_atr_mult=1.5,
        bb_length=20,
        bb_mult=2.0,
        min_volume_mult=1.2,
        trading_start_hour=9,
        trading_start_minute=30,
        trading_end_hour=11,
        trading_end_minute=30,
        close_exit_hour=15,
        close_exit_minute=15,
        avoid_lunch=True
    )

    def init(self):
        # Technical indicators
        self.macd = self.I(ta.macd, self.data.Close, fast=self.macd_fast, slow=self.macd_slow, signal=self.macd_signal)
        self.rsi = self.I(ta.rsi, self.data.Close, length=self.rsi_length)
        self.ema_9 = self.I(ta.ema, self.data.Close, length=self.ema_fast)
        self.ema_21 = self.I(ta.ema, self.data.Close, length=self.ema_slow)
        self.atr = self.I(ta.atr, self.data.High, self.data.Low, self.data.Close, length=self.atr_length)
        self.volume_sma = self.I(ta.sma, self.data.Volume, length=20)
        self.adx = self.I(ta.adx, self.data.High, self.data.Low, self.data.Close, length=14)

        # Trailing stop levels
        self.long_trail_stop = None
        self.short_trail_stop = None

    def in_time_window(self, time):
        p = self.params
        start = (time.hour > p['trading_start_hour'] or
                (time.hour == p['trading_start_hour'] and time.minute >= p['trading_start_minute']))
        end = (time.hour < p['trading_end_hour'] or
              (time.hour == p['trading_end_hour'] and time.minute < p['trading_end_minute']))
        return start and end

    def in_close_window(self, time):
        p = self.params
        return time.hour > p['close_exit_hour'] or \
               (time.hour == p['close_exit_hour'] and time.minute >= p['close_exit_minute'])

    def is_lunch(self, time):
        p = self.params
        return p['avoid_lunch'] and ((time.hour == 11 and time.minute >= 30) or (11 < time.hour < 13) or (time.hour == 13 and time.minute < 30))

    def next(self):
        p = self.params

        if len(self.data.Close) < max(p['macd_slow'], p['rsi_length'], p['ema_slow'], p['atr_length']):
            return

        time_now = pd.to_datetime(self.data.index[-1]).time()
        if not self.in_time_window(time_now) or self.is_lunch(time_now):
            return

        # Indicator values
        ema9 = self.ema_9[-1]
        ema21 = self.ema_21[-1]
        atr = self.atr[-1]
        atr_sma = pd.Series(self.atr).rolling(20).mean().iloc[-1]
        rsi_val = self.rsi[-1]
        adx_val = self.adx['ADX_14'][-1]
        volume_ok = self.data.Volume[-1] > self.volume_sma[-1] * p['min_volume_mult']
        volatility_ok = atr > atr_sma * 0.8
        trend_strength_ok = adx_val > 20

        # Entry logic
        macd_line = self.macd['MACD_12_26_9']
        macd_signal = self.macd['MACDs_12_26_9']
        macd_bullish = crossover(macd_line, macd_signal)
        macd_bearish = crossover(macd_signal, macd_line)

        uptrend = ema9 > ema21 and self.data.Close[-1] > ema9
        downtrend = ema9 < ema21 and self.data.Close[-1] < ema9

        long_condition = all([
            uptrend, macd_bullish, rsi_val < p['rsi_overbought'], volume_ok, volatility_ok, trend_strength_ok
        ])

        short_condition = all([
            downtrend, macd_bearish, rsi_val > p['rsi_oversold'], volume_ok, volatility_ok, trend_strength_ok
        ])

        daily_loss_limit = self.equity * (p['max_daily_loss'] / 100)
        daily_loss_exceeded = self.equity - self.initial_capital < -daily_loss_limit
        max_positions_reached = len(self.trades) >= p['max_positions']

        # Entry Execution
        if long_condition and not self.position and not daily_loss_exceeded and not max_positions_reached:
            stop = self.data.Close[-1] - (atr * p['atr_multiplier'])
            if stop >= self.data.Close[-1]: return
            risk_amt = self.equity * (p['risk_percent'] / 100)
            size = risk_amt / (self.data.Close[-1] - stop)
            tp = self.data.Close[-1] + (self.data.Close[-1] - stop) * p['reward_ratio']
            self.buy(sl=stop, tp=tp, size=size)

        elif short_condition and not self.position and not daily_loss_exceeded and not max_positions_reached:
            stop = self.data.Close[-1] + (atr * p['atr_multiplier'])
            if stop <= self.data.Close[-1]: return
            risk_amt = self.equity * (p['risk_percent'] / 100)
            size = risk_amt / (stop - self.data.Close[-1])
            tp = self.data.Close[-1] - (stop - self.data.Close[-1]) * p['reward_ratio']
            self.sell(sl=stop, tp=tp, size=size)

        # Trailing stops
        if self.position:
            trail = atr * p['trail_atr_mult']
            if self.position.is_long:
                new_stop = self.data.Close[-1] - trail
                if self.long_trail_stop is None or new_stop > self.long_trail_stop:
                    self.long_trail_stop = new_stop
                if self.data.Close[-1] <= self.long_trail_stop:
                    self.position.close()
            elif self.position.is_short:
                new_stop = self.data.Close[-1] + trail
                if self.short_trail_stop is None or new_stop < self.short_trail_stop:
                    self.short_trail_stop = new_stop
                if self.data.Close[-1] >= self.short_trail_stop:
                    self.position.close()

        if not self.position:
            self.long_trail_stop = None
            self.short_trail_stop = None

        # Time-based exit before market close
        if self.in_close_window(time_now) and self.position:
            self.position.close()
            self.long_trail_stop = None
            self.short_trail_stop = None

# 9. Random State Machine Strategy

# Improved Random State Machine Strategy

# 10. Reversal Trap Sniper

# Reversal Trap Sniper